# Example Jupyter notebook for running the eMERLIN CASA pipeline

This is an example Jupyter notebook for running the [eMERLIN CASA pipeline](https://github.com/e-merlin/CASA_eMERLIN_pipeline). It makes use of the [Jupyter kernel for CASA](https://github.com/rainsworth/jupyter-casa).

## 0. Setup

### eMERLIN CASA Pipeline

For general information and instructions, see the CASA eMERLIN pipeline on [GitHub](https://github.com/e-merlin/CASA_eMERLIN_pipeline).

You will need the version of the pipeline compatible with the Jupyter kernel for CASA, which is currently the [jupyter_notebook](https://github.com/rainsworth/CASA_eMERLIN_pipeline/tree/jupyter_notebook) branch on Rachael Ainsworth's fork.

If you have git installed, you can get the pipeline using:

```git clone -b jupyter_notebook https://github.com/rainsworth/CASA_eMERLIN_pipeline.git```

### Singularity

The [Singularity](http://singularity.lbl.gov/index.html) image can be downloaded by executing:

```singularity pull --name jupyter-casa.simg shub://aardk/jupyter-casa:docker```

The simplest way to start the Jupyter server is to execute:

```singularity run jupyter-casa.simg```

A Singularity containter runs with UID of the current user (i.e. the user executing ```singularity run```). The home directory of the user on the local filesystem will also be accessible inside the container, but by default only the home directory is shared with the container. Therefore any symbolic links which point to locations outside of the home directory will not be valid inside the container.

Fortunately, it is fairly straigthforward to make your local filesystem accessible to the container using the -B option. For example, to mount a local directory called /data inside the container, execute:

```singularity run -B /data:$HOME/data jupyter-casa.simg```

To mount the pipeline directory cloned above and the directory with the data you wish to run the pipeline on:

```singularity run -B /CASA_eMERLIN_pipeline:$HOME/pipeline -B /data:$HOME/data jupyter-casa.simg```

Copy and paste the Jupyter Notebook token provided into your browser.

### Docker

Follow [these](https://github.com/aardk/jupyter-casa) instructions. 

If on a Mac (from [this](https://fredrikaverpil.github.io/2016/07/31/docker-for-mac-and-gui-applications/) blog post):

~~~~
> open -a XQuartz
> ip=$(ifconfig en0 | grep inet | awk '$1=="inet" {print $2}')
> xhost + $ip
> docker run --rm -p 8888:8888 -i -t -v /tmp/.X11-unix:/tmp/.X11-unix -e DISPLAY=$ip:0 -v /local/path/to/pipeline/:/home/jupyter/pipeline -v /local/path/to/data/:/home/jupyter/data penngwyn/jupytercasa /bin/sh -c "jupyter notebook"
~~~~

Copy and paste the Jupyter Notebook token provided into your browser.



## 1. How to run the pipeline

To execute the pipeline from within this notebook:
~~~~
> run_in_casa = True
> pipeline_path = '/path/to/pipeline_path/'   # You need to define this variable explicitly
> execfile(pipeline_path + 'eMERLIN_CASA_pipeline.py')
> run_pipeline(inputs)
~~~~

In [1]:
casa['build']['version']

'5.1.1-5  '

In [2]:
import sys

# singularity module paths:
#sys.path.append('/home/rainswor/pipeline/')
#sys.path.append('/home/rainswor/pipeline/functions/')

# docker module paths:
sys.path.append('/home/jupyter/pipeline/')
sys.path.append('/home/jupyter/pipeline/functions/')

In [3]:
run_in_casa = True

In [4]:
# singularity pipeline path:
#pipeline_path = '/home/rainswor/pipeline/'

# docker pipeline path
pipeline_path = '/home/jupyter/pipeline/'

In [5]:
execfile(pipeline_path + 'eMERLIN_CASA_pipeline.py')

Pipeline initialized. To run the pipeline within CASA use:
inputs, caltables, msinfo = run_pipeline(inputs_path=<input file>)



## 2. How to reduce e-MERLIN data

### Data preparation
- Download the pipeline (it can be in any location).
- Download the fits-IDI files from the observatory (they can be in any unique location).
- Create the working path where you will work, and copy the `inputs.txt` to your working path.
- Edit the inputs.txt file: fill the `fits_path` (where the fits files are located) and the `inbase` (any name you want to give to your project).
- Leave all other parameters as default, and only use the steps `run_importfits=1` and `summary_weblog=1`.
- Data will be converted to MS and prepared. Open in a web browser the file `./weblog/index.html`.
- Check the listobs file in the 'Observation summary' tab and fill in the inputs.txt file the fields `targets`, `phscals`, `fluxcal`,`bpcal`,`ptcal`.
- Select a reference antenna `refant`. If you are not sure set `refant=''` and rerun the `summary_weblog` step alone. The pipeline will try to suggest a list of the best reference antennas to use.
- Run the rest of the pre-processing steps depending on your needs. If you have a list of manual flags to apply, set `flag_2a_manual=1` and remember to specify where that file is by setting `manual_flags_a` in the user inputs.
- In most cases it is recommended to run `average_1=1` to split the date to a new averaged dataset `inbase_avg.ms`.

### Data calibration
- Prepare the file `manual_flags_b` with flags commands based on the plots produced in the previous section (or your own data exploration). The external file should be set in `manual_flags_b` in the user inputs.
- Run all the calibration steps by setting them to 1. You may prefer to run each of them one by one and check the output plots
- You can always select the step `weblog=1`. It will just update the weblog with any new plots available.
- Improve the `manual_flags_a` file to have more detailed flagging as you proceed with the calibration.
- It is a good practice to redo the calibration once you are happy with your flags and you are sure of all the steps. For that, you can rerun `average_1` to produce from scratch the `inbase_avg.ms` dataset and repeat all the calibration steps.


## 3. Inputs

There are two types of inputs: the **user inputs** and the **process inputs**. The user inputs expect a string that will depend on the project, sources, antennas and external files. The process inputs expect an integer that can be 0 to not run a step, or 1 to run it. Additionally, steps that produce calibration tables can be set to also apply the calibration to the data modifying the corrected column. So for `bandpass_0`, `delay`, `gain_0_p_ap`, `fluxscale`, `bandpass_1_sp` and `gain_1_amp_sp`, a value of 2 means run the step and apply the calibration up to that step. This is useful to check the calibration up to each step, but you can also use `applycal_all=1` to apply everything when all tables are produced.


In [6]:
# default values for inputs from CASA eMERLIN pipeline

inputs = {

  # User inputs
  "fits_path": "/home/jupyter/data/",           # [str] Path to the location of the fits files. Can be an absolute or relative path. A single directory must be specified. All fits and FITS files from that directory are considered.
  "inbase": "project_name",                     # [str] Project name. It will be used to give names to the MS, plots and tables.
  "targets": "",                                # [str] Names of sources as they appear in the MS to be used as targets. Can be a comma-separated string. If more than one target is selected, the corresponding phase calibrator has to keep the same order.
  "phscals": "",                                # [str] Names of sources as they appear in the MS to be used as calibrators. Can be a comma-separated string. If more than one phase calibrator is selected, the order needs to match the targets. If the same phase calibrator is used for different targets, just repeat the name as many times as needed.
  "fluxcal": "1331+305",                        # [str] Name of source as it appears in the MS to be used as flux calibrator. Only one source accepted. 1331+305 expected. If a different source is selected the fluxscale step will not work properly.                            
  "bpcal": "1407+284",                          # [str] Names of sources as they appear in the MS to be used as calibrators. Can be a comma-separated string.
  "ptcal": "0319+415",                          # [str] Point-like calibrator. Names of sources as they appear in the MS to be included in calibration steps, but not used to calibrate other sources by now. For the moment, consider it for check sources. 
  "refant": "Mk2",                              # [str] Antenna name to be used as reference antenna. Can accept a comma-separated string, but CASA can only manage a single value for now (there are plans to allow gaincal to use a prioritized list. If empty, the pipeline will try to search for the most suitable antennas. 
  "Lo_dropout_scans": "",                       # [str] Comma-separated list of scans in which the Lo telescope was not observing the phase calibrator(s). This has two effects: those scans will be flagged in the flag_1_apriori step, and also each calibration table created will be edited to remove solutions for antenna Lo for the phase calibrator scans listed in Lo_dropout_scans. Example: Lo_dropout_scans = '4,8,12,16,20,24,28'
  "manual_flags_a": "./inputfg_a.flags",        # [str] Path to an external file that contains a list of flag commands, one per line. The flags will be applied to the unaveraged dataset inbase.ms when step flag_2a_manual is enabled. The file will be read by CASA task flagdata using mode='list' and inpfile will be set to manual_flags_a. Note from the CASA documentation (https://casa.nrao.edu/casadocs/casa-5.1.1/global-task-list/task_flagdata/about) on the format of the file: There should be no whitespace between KEY=VALUE since the parser first breaks command lines on whitespace, then on "=". Use only one whitespace to separate the parameters (no commas).
  "manual_flags_b": "./inputfg_b_avg.flags",    # [str] Same as manual_flags_a but will be applied to the averaged data set inbase_avg.ms when step flag_2b_manual is enabled.
    
  # Pre-processing (0=don't run, 1=run)
  "run_importfits": 0,  
  "summary_weblog": 0,
  "hanning": 0,
  "ms2mms": 0,
  "flag_0_aoflagger": 0,
  "flag_1_apriori": 0,
  "flag_2a_manual": 0,
  "shift_field_pos": 0,
  "average_1": 0,
  "plot_data": 0,
  "save_flags": 0,
  "restore_flags": 0,   
  
  # Calibration (0=don't run, 1=run)
  "flag_2b_manual": 0,
  "init_models": 0,
  "bandpass_0": 0,
  "flag_3_tfcropBP": 0,
  "delay": 0,
  "gain_0_p_ap": 0,
  "fluxscale": 0,
  "bandpass_1_sp": 0,
  "gain_1_amp_sp": 0,
  "applycal_all": 0,
  "flag_4_rflag": 0,
  "plot_corrected": 0,
  "first_images": 0,
  "flag_statistics": 0,
  "weblog": 0
}

                


## 4. Procedures


## 4.1 Pre-processing

### 4.1.1 run_importfitsidi
Merge fits-IDI files in `fits_path` to form an MS named `inbase.ms`

Inputs parameters needed:
```
fits_path               [str]
inbase                  [str]
```

Output:
```
inbase.ms               [MS]
inbase.ms.listobs.txt   [txt]
inbase.ms.sp0           [MS] (only for mixed mode observations)
```

First, `inbase.ms` is removed if present (`inbase` is the parameter in inputs file). All files ending in .fits or .FITS in the folder `fits_path` are considered for importing and will be merged. The logger shows which files have been found. Several CASA tasks are executed: `importfitsidi` is executed with `constobsid=True, scanreindexgap_s=15.0`. The dataset can be averaged in time at this point by setting the `run_importfits = n` with n > 1. The dataset will be averaged with a timebin of `n`. 

Then `uvfix` with option `reuse=False` is run to correct UVW values and the output MS replaces `inbase.ms`. Then `flagdata` with `mode='manual',autocorr=True` to remove autocorrelations. Finally, `listobs` is run and a new listobs file is created in `inbase.ms.listobs.txt`.

At this point, the pipeline checks if the observations were observed in mixed mode. If so, the spectral spws will be split and `inbase.ms` will contain only the continuum data (broadband, low spectral resolution). The high resolution spw will be splitted in new MS ending with sp0, sp1, etc.

Run the pipeline:

In [7]:
# In the case below, we are using the tutorial data from https://github.com/hrampadarath/emerlin-casa-jupyter 
# to compare our outputs. The data is already in MS format and averaged, so we can skip the run_importfitsidi step. 
# Our inputs for this step are therefore:

inputs["fits_path"] = "/home/jupyter/data/"         # [str] Path to the location of the fits files. Can be an absolute or relative path. A single directory must be specified. All fits and FITS files from that directory are considered.
inputs["inbase"] = "all_avg"                        # [str] Project name. It will be used to give names to the MS, plots and tables.


#inputs["run_importfits"] = 1
#run_pipeline(inputs=inputs)

### 4.1.2 summary_weblog
Retrieve basic information from the original MS, produce some plots and compile everything in the weblog.

Inputs parameters needed:
```
inbase                  [str]
```

Output:
```
inbase.ms.msinfo.pkl    [pickle dictionary]
```
Runs `get_msinfo` (see section 3, below) to get information from the inputs file and directly from the MS. It prepares plots for elevation vs time and uvcov for each individual sources. It produces a weblog with the available information. If no `refant` is selected, or the antenna selected is not in the MS, a procedure will try to guess a good refant to use. Please update the inputs file with a suitable refant.

Edit inputs and run the pipeline:

In [8]:
inputs["run_importfits"] = 0
inputs["summary_weblog"] = 1

run_pipeline(inputs=inputs)

2018-07-09 12:04:37 | INFO | Starting pipeline
2018-07-09 12:04:37 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:04:37 | INFO | Pipeline version: v0.7.11
2018-07-09 12:04:37 | INFO | Using github branch: jupyter_notebook
2018-07-09 12:04:37 | INFO | github last commit: c145017
2018-07-09 12:04:37 | INFO | This log uses UTC times
2018-07-09 12:04:37 | INFO | Loaded previous flags list from: ./flags.pkl
2018-07-09 12:04:37 | INFO | Using MS file: all_avg.ms
2018-07-09 12:04:37 | INFO | Reading ms file information for MS: all_avg.ms
2018-07-09 12:04:37 | INFO | Targets:   
2018-07-09 12:04:37 | INFO | Phasecals: 
2018-07-09 12:04:37 | INFO | Fluxcal:   1331+305
2018-07-09 12:04:37 | INFO | Bandpass:  1407+284
2018-07-09 12:04:37 | INFO | Pointcal:  0319+415
2018-07-09 12:04:37 | INFO | Sources in MS all_avg.ms: 1302+5748,1252+5634,1331+305,1407+284,0319+415
2018-07-09 12:04:38 | INFO | Antennas in MS all_avg.ms: ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm']
2018-07-09 12:0

({'Lo_dropout_scans': '',
  'applycal_all': 0,
  'average_1': 0,
  'bandpass_0': 0,
  'bandpass_1_sp': 0,
  'bpcal': '1407+284',
  'delay': 0,
  'first_images': 0,
  'fits_path': '/home/jupyter/data/',
  'flag_0_aoflagger': 0,
  'flag_1_apriori': 0,
  'flag_2a_manual': 0,
  'flag_2b_manual': 0,
  'flag_3_tfcropBP': 0,
  'flag_4_rflag': 0,
  'flag_statistics': 0,
  'fluxcal': '1331+305',
  'fluxscale': 0,
  'gain_0_p_ap': 0,
  'gain_1_amp_sp': 0,
  'hanning': 0,
  'inbase': 'all_avg',
  'init_models': 0,
  'manual_flags_a': './inputfg_a.flags',
  'manual_flags_b': './inputfg_b_avg.flags',
  'ms2mms': 0,
  'phscals': '',
  'plot_corrected': 0,
  'plot_data': 0,
  'ptcal': '0319+415',
  'refant': 'Mk2',
  'restore_flags': 0,
  'run_importfits': 0,
  'save_flags': 0,
  'shift_field_pos': 0,
  'summary_weblog': 1,
  'targets': '',
  'weblog': 0},
 {'antennas': ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm'],
  'band': 'C',
  'baselines': ['Mk2&Kn',
   'Mk2&De',
   'Mk2&Pi',
   'Mk2&Da',
   'Mk2&Cm',


View the summary weblog [here](http://localhost:8888/view/data/weblog/index.html).

Check the listobs file in the 'Observation summary' tab and fill in the `inputs` fields `targets`, `phscals`, `fluxcal`, `bpcal` and `ptcal`.

Select a reference antenna `refant`. If you are not sure set `refant=''` and rerun the `summary_weblog` step alone. The pipeline will try to suggest a list of the best reference antennas to use.

Run the rest of the pre-processing steps depending on your needs. If you have a list of manual flags to apply, set `flag_2a_manual=1` and remember to specify where that file is by setting `manual_flags_a` in the user inputs.

In most cases it is recommended to run `average_1=1` to split the data to a new averaged dataset `inbase_avg.ms`.

In [9]:
inputs["targets"] = "1252+5634"                     # [str] Names of sources as they appear in the MS to be used as targets. Can be a comma-separated string. If more than one target is selected, the corresponding phase calibrator has to keep the same order.
inputs["phscals"] = "1302+5748"                     # [str] Names of sources as they appear in the MS to be used as calibrators. Can be a comma-separated string. If more than one phase calibrator is selected, the order needs to match the targets. If the same phase calibrator is used for different targets, just repeat the name as many times as needed.
# leave rest of inputs as default


### 4.1.3 hanning

Just runs CASA hanning smoothing.

Inputs parameters needed:
```
inbase                  [str]
```

Output:
```
inbase.ms               [MS]
```

Runs mstransform with mode='hanning' on DATA column. It produces a new MS, which substitutes the `inbase.ms`. It works with MS or MMS. This step is optional but recommended for L band observations. It is probably not needed for C band datasets. A value of 1 will check the band of the dataset, and apply Hanning smoothing only if it is an L band dataset. A value higher than 1 will force Hanning smoothing, regardless of the band.

In [10]:
inputs["summary_weblog"] = 0
#inputs["hanning"] = 1

#run_pipeline(inputs=inputs)

### 4.1.4 ms2mms
Optional step to transform original MS to an MMS.

Inputs parameters needed:
```
inbase                  [str]
```

Output:
```
inbase.mms               [MMS]
```

This allows CASA to run tasks on the MultiMeasurementSet in parallel transparently. Takes time to convert but later makes calibration and imaging faster. To be tested in detail. Uses CASA task `partition` with default parameters. **Important**: if ms2mms is executed, it will delete `inbase.ms` and all the following steps will be run on the new `inbase.mms`.



In [11]:
#inputs["hanning"] = 0
#inputs["ms2mms"] = 1

#run_pipeline(inputs=inputs)

### 4.1.5 flag_0_aoflagger
Uses aoflagger to autoflag data using predefined strategies

Inputs parameters needed:
```
inbase                  [str]
```

It runs [aoflagger](https://sourceforge.net/p/aoflagger/wiki/Home/) on all the fields in the MS. aoflagger v2.9+ required. The pipeline might work with previous versions in some situations, but it is not recommended and probably will fail The strategies are selected by field following this criteria: (1) First, check if the user has a new strategy for this field in the local folder ./aoflagger_strategies/user/, the strategy needs to match: `<field>.rfis`. (2) If not, check if the user has produced a new strategy for this field in the pipeline folder. So will search for `pipeline_path+'aoflagger_strategies/default/<field>.rfis`, (3) If nothing is found, just use the default strategy. For example for field 1234+5678 the prioritization for searching strategies is:

 1. ./aoflagger_strategies/user/1234+5678.rfis
 2. /path/to/pipeline/aoflagger_strategies/default/1234+5678.rfis
 3. /path/to/pipeline/aoflagger_strategies/default/default_faint.rfis

Two values are accepted for this step. `flag_0_aoflagger=1` means to process each spw separately and `flag_0_aoflagger=2` means to compute all spw at the same time. The flags produced are the same, the only difference is that 2 will read data from all the spw at the same time. 2 can be used when the data set is small (about 50GB or less, but depends on free memory) because you save the overhead of restarting aoflagger once per spw (and per source). 1 can be used for large datasets, as it does not require so much memory and will run much faster, even if many loops are needed.


In [12]:
#inputs["ms2mms"] = 0
#inputs["flag_0_aoflagger"] = 1

#run_pipeline(inputs=inputs)

### 4.1.6 flag_1_apriori
Applies a-priori standard flags.

Inputs parameters needed:
```
inbase                  [str]
targets                 [str]
phscals                 [str]
fluxcal                 [str]
bpcal                   [str]
ptcal                   [str]
Lo_dropout_scans        [str]
```
Different flags are applied to the data:

 - Lo&Mk2 for all sources.
 - Edge channels for all sources, all spw, defined as spw='*:0~(nchan/128-1);(nchan-nchan/128)~(nchan-1)'.
 - Quack 20 seconds on all sources in the MS.
 - Quack for 5 minutes for 1331+305, 1407+284, 0319+415 if they are present in the MS (no need to include in inputs file.
 - Quack n seconds for all targets and phasecals. The number of seconds depend on the target-phasecal separation in degrees. 20s for separation < 1deg, 25s for 1.0 <= separation < 2.0, 30s for 2.0 <= separation < 3.5, 35s for separation >= 3.5.
 - Scans in `Lo_dropout_scans` for antenna = 'Lo' and fields in `phscals`.

Source names in the inputs parameters are only needed in two cases: `phscals` in order to flag Lo_dropout scans, and `targets`+`phscals` to compute quackintervals that depend on the target-phasecal separations (otherwise 20s are flagged by default). The rest of the a-priori flagging steps do not require any source name in the inputs file.


In [13]:
#inputs["flag_0_aoflagger"] = 0
#inputs["flag_1_apriori"] = 1

#run_pipeline(inputs=inputs)

### 4.1.7 flag_2a_manual
Applies flags from an external file with a list of flag commands the unaveraged dataset.

Inputs parameters needed:

```
inbase                  [str]
manual_flags_a          [str, path to file]
```

Simply runs `flagdata(vis='inbase.ms', mode='list', inpfile=manual_flags_a)`. This is run on the original, unaveraged, dataset `inbase.ms`. To apply manual flags to an averaged dataset see `flag_2b_manual` below.

 For more information on this flagdata mode see [flagdata](https://casa.nrao.edu/casadocs/casa-5.1.1/global-task-list/task_flagdata/about). For details on the format of the file see [examples](https://casa.nrao.edu/casadocs/casa-5.1.1/global-task-list/task_flagdata/examples). It needs one flag command per line, with parameters separated by one space (don't use commas!). For example:

```
mode='manual' field='1331+305' antenna='' timerange='10:00:00~10:11:30'
mode='manual' field='' antenna='' timerange='' spw='0:0~30'
mode='manual' field='' antenna='Mk2' timerange='09:05:00~16:27:00'
mode='manual' field='1258-2219' antenna='' timerange='12:57:01~12:59:59'
mode='quack' field='1258-2219,1309-2322' quackinterval=24.
```

In [14]:
#inputs["flag_1_apriori"] = 0
#inputs["flag_2a_manual"] = 1

#run_pipeline(inputs=inputs)

### 4.1.8 shift_field_pos
Generate new field positions by shifting the phase center.

Inputs parameters needed:
```
inbase                  [str]
shift_phasecenter.txt   [external file]
```

Output:
```
inbase.ms               [MS]
inbase.ms.listobs.txt   [txt]
```

The text file shift_phasecenter.txt should contain one shift per line, with three comma-separated values each: field, new field name, and new position coordinates. For example:

```
0336+3218, 0336+3218_pos1, J2000 03h36m30.10s +32d18m30.0s
0336+3218, 0336+3218_pos2, J2000 03h36m30.10s +32d18m40.0s
0332+3205, 0336+3218_pos1, J2000 03h32m28.30s +32d05m46.0s
```

Spaces will be ignored except inside the coordinate string. That file will produce two shifts of field 0336+3218 and one shift of field 0332+3205 to the corresponding positions indicated. For each shift, the field will be splitted in a temporary MS on which the shift will be computed. Then that temporary MS will be merged to the main data set. WARNING1: this step will add new fields to the MS (using concat) without removing any previous field: this adds phase centers, does not update the original ones. That means that the resulting MS will have two or more **new** fields with the same time stamps and same scan number but with different phase centers. This task is not intended to correct the phase center of a field, as it will keep the original one. However, you can always choose only the relevant fields when averaging the data in step `average_1`, for example you may want only corrected positions 0336+3218_pos2 and 0336+3218_pos1. WARNING2: note that because new fields are added to the MS, if you want to use them you need to specify them in the inputs file.

NOTE: Apparently CASA cannot concatenate (`concat`) into a MMS. Therefore `ms2mms` is not compatible with `shift_field_pos`. If you want to use a MMS and shift a phase center your options are: (a) run `ms2mms` only after producing the shift, (b) modify `shift_field_pos` code to use `virtualconcat` instead of `concat`, (c) ignore that and do the phase shifts manually.


In [15]:
#inputs["flag_2a_manual"] = 0
#inputs["shift_field_pos"] = 1

#run_pipeline(inputs=inputs)

### 4.1.9 average_1
Split dataset and average to reduce data volume.

Inputs parameters needed:
```
inbase                  [str]
targets                 [str]
phscals                 [str]
fluxcal                 [str]
bpcal                   [str]
ptcal                   [str]
```

Output:
```
inbase_avg.ms               [MS]
inbase_avg.ms.listobs.txt   [txt]
```

It will create a new MS `inbase_avg.ms` (or inbase_avg.mms if working with mms files). It will remove previous inbase_avg.ms file if it exists. Only the fields selected in the inputs file will be included in inbase_avg.ms. Data is averaged using `width=4` and `timebin='1s'` (This will be `2s` when CASA gaincal fixes its bug related to VisibilityIterator2). Optionally you can use a value different than `1` in the inputs file, and that number will be the timebin value in seconds. `keepflags=False` in this version.

---
**Important** when the inbase_avg.ms file is produced, all the steps of the pipeline after this one will always work on inbase_avg.ms only, and not the original dataset. At this point, `get_msinfo` is executed again, and the `msinfo` dictionary is created and saved in `inbase_avg.ms.msinfo.pkl`. If the averaged dataset does not exist, the pipeline will always use the unaveraged dataset `inbase.ms`.


In [16]:
#inputs["shift_field_pos"] = 0
#inputs["average_1"] = 1

#run_pipeline(inputs=inputs)

### 4.1.10 plot_data
Produce plots of amp/phase vs time/freq for each baseline	plotms

Inputs parameters needed:
```
inbase                  [str]
targets                 [str]
phscals                 [str]
fluxcal                 [str]
bpcal                   [str]
ptcal                   [str]
```

Output:
```
./plots/plots_data      [directory with multiple plots in png format]
```

Produces plots in png format for the visibilities in `inbase_avg.ms`. It iterates through all sources specified by the user in the inputs file, and produces plots for each baseline. All plots are stored in the output directory. The plots have some averaging:

 - Amp/Phase vs Time: all channels averaged in each spw, colorized by spw.
 - Amp/Phase vs Frequency: averaged every 5 min, colorized by correlation.


In [17]:
#inputs["average_1"] = 0
inputs["plot_data"] = 1

run_pipeline(inputs=inputs)

2018-07-09 12:04:50 | INFO | Starting pipeline
2018-07-09 12:04:50 | INFO | Starting pipeline
2018-07-09 12:04:50 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:04:50 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:04:50 | INFO | Pipeline version: v0.7.11
2018-07-09 12:04:50 | INFO | Pipeline version: v0.7.11
2018-07-09 12:04:50 | INFO | Using github branch: jupyter_notebook
2018-07-09 12:04:50 | INFO | Using github branch: jupyter_notebook
2018-07-09 12:04:50 | INFO | github last commit: c145017
2018-07-09 12:04:50 | INFO | github last commit: c145017
2018-07-09 12:04:50 | INFO | This log uses UTC times
2018-07-09 12:04:50 | INFO | This log uses UTC times
2018-07-09 12:04:50 | INFO | Loaded previous flags list from: ./flags.pkl
2018-07-09 12:04:50 | INFO | Loaded previous flags list from: ./flags.pkl
2018-07-09 12:04:50 | INFO | Using MS file: all_avg.ms
2018-07-09 12:04:50 | INFO | Using MS file: all_avg.ms
2018-07-09 12:04:50 | INFO | Rea

({'Lo_dropout_scans': '',
  'applycal_all': 0,
  'average_1': 0,
  'bandpass_0': 0,
  'bandpass_1_sp': 0,
  'bpcal': '1407+284',
  'delay': 0,
  'first_images': 0,
  'fits_path': '/home/jupyter/data/',
  'flag_0_aoflagger': 0,
  'flag_1_apriori': 0,
  'flag_2a_manual': 0,
  'flag_2b_manual': 0,
  'flag_3_tfcropBP': 0,
  'flag_4_rflag': 0,
  'flag_statistics': 0,
  'fluxcal': '1331+305',
  'fluxscale': 0,
  'gain_0_p_ap': 0,
  'gain_1_amp_sp': 0,
  'hanning': 0,
  'inbase': 'all_avg',
  'init_models': 0,
  'manual_flags_a': './inputfg_a.flags',
  'manual_flags_b': './inputfg_b_avg.flags',
  'ms2mms': 0,
  'phscals': '1302+5748',
  'plot_corrected': 0,
  'plot_data': 1,
  'ptcal': '0319+415',
  'refant': 'Mk2',
  'restore_flags': 0,
  'run_importfits': 0,
  'save_flags': 0,
  'shift_field_pos': 0,
  'summary_weblog': 0,
  'targets': '1252+5634',
  'weblog': 0},
 {'antennas': ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm'],
  'band': 'C',
  'baselines': ['Mk2&Kn',
   'Mk2&De',
   'Mk2&Pi',
   'Mk2&

### 4.1.11 save_flags
Saves the current status of the flagging to the flag table with versionname='initialize_flags'.

Output:
```
initialize_flags        [flag table]
```

Makes a copy of the current flags to the flag table with versionname `initialize_flags` using flagmanager. It will overwrite previous versions of that table. This table means to contain flags produced by aoflagger, a-priori and manual_a flags, but only if the user has saved this table after producing those flags (so it is the responsability of the user to keep track of what was applied when this step is executed).


In [18]:
inputs["plot_data"] = 0
#inputs["save_flags"] = 1

#run_pipeline(inputs=inputs)

## 4.2. Calibration

All calibration steps require the following input parameters:
```
inbase                  [str]
targets                 [str]
phscals                 [str]
fluxcal                 [str]
bpcal                   [str]
ptcal                   [str]
refant                  [str]
Lo_dropout_scans        [str]
```

You can select which steps to run in the inputs file by setting the corresponding number to:

 - 0: don't run the step
 - 1: run the step and produce calibration tables and plots
 - 2: run the step and produce calibration tables and plots and apply the calibration to the data.

During calibration, solutions are found using only the inner ~90% channels of each spw. So `innerchan=0.1*(nchan-nchan/512.), 0.9*(nchan-nchan/512.)`.

**Important.** Below we list the parameters used to create each table in each calibration step. `previous_cal` is a list of names of previous calibration tables that will be used to fill the `gaintable`, `gainfield`, `interp`, `spwmap`. Those parameters are defined when the table is created, and they will be used only to generate other tables when the corresponding table name is listed in `previous_cal`. To know the values of those parameters you have to check them in the description of the corresponding table.

The step applycal_all will apply all calibration: it will write `CORRECTED_DATA` column in the MS using the last caltables. In particular it will use `delay.K1,bpcal_sp.B1,allcal_p.G0,allcal_p_jitter.G0,allcal_ap.G3` for the calibrators and `delay.K1,bpcal_sp.B1,phscal_p_scan.G2,allcal_ap_scan.G3` for the targets. If you want to apply the calibration up to an intermediate step (for instance you want to apply the delay and BP caltables only to check their effects before proceeding), you can set in the inputs file a value of `2`, which means apply all previous calibration up to that step. Below you can find the **applycal** description at the end of each step that shows which tables will be applied to calibrators and targets respectively if `2` is selected for that step in the inputs file.

`calsources` is a comma-separated list of all calibrator sources.

### 4.2.1 restore_flags
Restore the status of the flagging that was saved in the flag table with versionname='initialize_flags'.

Output:
```
initialize_flags        [flag table]
```

Restores the flags saved in the flag table with versionname `initialize_flags` using flagmanager. The idea is to use this saving point to restart calibration from scratch but keeping the flags produced during the pre-process stage. Any other tables associated with the dataset will not be removed, so use flagmanager and remove them manually if needed.



In [19]:
#inputs["save_flags"] = 0
#inputs["restore_flags"] = 1

#run_pipeline(inputs=inputs)

### 4.2.2 flag_2b_manual
Applies flags from an external file with a list of flag commands to the averaged dataset.

Inputs parameters needed:

```
manual_flags_b          [str, path to file]
```

Simply runs `flagdata(vis='inbase_avg.ms', mode='list', inpfile=manual_flags_b)`. This is run on the averaged dataset `inbase_avg.ms`. To apply manual flags to an averaged dataset see `flag_2b_manual` below.

For more information on this flagdata mode see [flagdata](https://casa.nrao.edu/casadocs/casa-5.1.1/global-task-list/task_flagdata/about). For details on the format of the file see [examples](https://casa.nrao.edu/casadocs/casa-5.1.1/global-task-list/task_flagdata/examples). It needs one flag command per line, with parameters separated by one space (don't use commas!). For example:

```
mode='manual' field='1331+305' antenna='' timerange='10:00:00~10:11:30'
mode='manual' field='' antenna='' timerange='' spw='0:0~30'
mode='manual' field='' antenna='Mk2' timerange='09:05:00~16:27:00'
mode='manual' field='1258-2219' antenna='' timerange='12:57:01~12:59:59'
mode='quack' field='1258-2219,1309-2322' quackinterval=24.
```

In [20]:
#inputs["restore_flags"] = 0
inputs["flag_2b_manual"] = 1
inputs["manual_flags_b"] = "./all_avg_1.flags"

run_pipeline(inputs=inputs)

2018-07-09 12:08:37 | INFO | Starting pipeline
2018-07-09 12:08:37 | INFO | Starting pipeline
2018-07-09 12:08:37 | INFO | Starting pipeline
2018-07-09 12:08:37 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:08:37 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:08:37 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:08:37 | INFO | Pipeline version: v0.7.11
2018-07-09 12:08:37 | INFO | Pipeline version: v0.7.11
2018-07-09 12:08:37 | INFO | Pipeline version: v0.7.11
2018-07-09 12:08:37 | INFO | Using github branch: jupyter_notebook
2018-07-09 12:08:37 | INFO | Using github branch: jupyter_notebook
2018-07-09 12:08:37 | INFO | Using github branch: jupyter_notebook
2018-07-09 12:08:37 | INFO | github last commit: c145017
2018-07-09 12:08:37 | INFO | github last commit: c145017
2018-07-09 12:08:37 | INFO | github last commit: c145017
2018-07-09 12:08:37 | INFO | This log uses UTC times
2018-07-09 12:08:37 | INFO | This log use

({'Lo_dropout_scans': '',
  'applycal_all': 0,
  'average_1': 0,
  'bandpass_0': 0,
  'bandpass_1_sp': 0,
  'bpcal': '1407+284',
  'delay': 0,
  'first_images': 0,
  'fits_path': '/home/jupyter/data/',
  'flag_0_aoflagger': 0,
  'flag_1_apriori': 0,
  'flag_2a_manual': 0,
  'flag_2b_manual': 1,
  'flag_3_tfcropBP': 0,
  'flag_4_rflag': 0,
  'flag_statistics': 0,
  'fluxcal': '1331+305',
  'fluxscale': 0,
  'gain_0_p_ap': 0,
  'gain_1_amp_sp': 0,
  'hanning': 0,
  'inbase': 'all_avg',
  'init_models': 0,
  'manual_flags_a': './inputfg_a.flags',
  'manual_flags_b': './all_avg_1.flags',
  'ms2mms': 0,
  'phscals': '1302+5748',
  'plot_corrected': 0,
  'plot_data': 0,
  'ptcal': '0319+415',
  'refant': 'Mk2',
  'restore_flags': 0,
  'run_importfits': 0,
  'save_flags': 0,
  'shift_field_pos': 0,
  'summary_weblog': 0,
  'targets': '1252+5634',
  'weblog': 0},
 {'antennas': ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm'],
  'band': 'C',
  'baselines': ['Mk2&Kn',
   'Mk2&De',
   'Mk2&Pi',
   'Mk2&Da',

### 4.2.3 init_models

The pipeline will initialize the model column for all sources different from 1331+305 using CASA `delmod`, so setting all amplitudes to 1 and all phases to 0 in the model column. For 1331+305 it will check if the data is L band or C band, and use `setjy` to introduce the correct model of 1331+305 (3C286) into the data column. The models can be found in `pipeline_path+'calibrator_models/'`. Only C and L band models available.

In [21]:
inputs["flag_2b_manual"] = 0
inputs["init_models"] = 1

run_pipeline(inputs=inputs)

2018-07-09 12:09:02 | INFO | Starting pipeline
2018-07-09 12:09:02 | INFO | Starting pipeline
2018-07-09 12:09:02 | INFO | Starting pipeline
2018-07-09 12:09:02 | INFO | Starting pipeline
2018-07-09 12:09:02 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:09:02 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:09:02 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:09:02 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:09:02 | INFO | Pipeline version: v0.7.11
2018-07-09 12:09:02 | INFO | Pipeline version: v0.7.11
2018-07-09 12:09:02 | INFO | Pipeline version: v0.7.11
2018-07-09 12:09:02 | INFO | Pipeline version: v0.7.11
2018-07-09 12:09:02 | INFO | Using github branch: jupyter_notebook
2018-07-09 12:09:02 | INFO | Using github branch: jupyter_notebook
2018-07-09 12:09:02 | INFO | Using github branch: jupyter_notebook
2018-07-09 12:09:02 | INFO | Using github branch: jupyter_notebook
2018-07-09 12:09:02 

({'Lo_dropout_scans': '',
  'applycal_all': 0,
  'average_1': 0,
  'bandpass_0': 0,
  'bandpass_1_sp': 0,
  'bpcal': '1407+284',
  'delay': 0,
  'first_images': 0,
  'fits_path': '/home/jupyter/data/',
  'flag_0_aoflagger': 0,
  'flag_1_apriori': 0,
  'flag_2a_manual': 0,
  'flag_2b_manual': 0,
  'flag_3_tfcropBP': 0,
  'flag_4_rflag': 0,
  'flag_statistics': 0,
  'fluxcal': '1331+305',
  'fluxscale': 0,
  'gain_0_p_ap': 0,
  'gain_1_amp_sp': 0,
  'hanning': 0,
  'inbase': 'all_avg',
  'init_models': 1,
  'manual_flags_a': './inputfg_a.flags',
  'manual_flags_b': './all_avg_1.flags',
  'ms2mms': 0,
  'phscals': '1302+5748',
  'plot_corrected': 0,
  'plot_data': 0,
  'ptcal': '0319+415',
  'refant': 'Mk2',
  'restore_flags': 0,
  'run_importfits': 0,
  'save_flags': 0,
  'shift_field_pos': 0,
  'summary_weblog': 0,
  'targets': '1252+5634',
  'weblog': 0},
 {'antennas': ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm'],
  'band': 'C',
  'baselines': ['Mk2&Kn',
   'Mk2&De',
   'Mk2&Pi',
   'Mk2&Da',

### 4.2.4 bandpass_0

It runs a delay, phase, and a&p calibration before finding the combined BP table for all sources listed in `bpcals`.

**bpcal_d.K0** - Delay calibration of bandpass calibrator(s)

|Table parameter| Value                    |
|---------------| -------------------------|
| name          | bpcal_d.K0               |
| field         | bpcal                    |
| previous_cal  | []                       |
| solint        | 180s                     |
| gaintype      | K                        |
| calmode       | p                        |
| spw           | *:innerchan              |
| combine       | spw                      |
| table         | ./calib/inbase_bpcal_d.K0|
| gainfield     | bpcal                    |
| interp        | linear                   |
| spwmap        | [0]*num_spw              |



**bpcal_p.G0** - Phase calibration of bandpass calibrator(s)

|Table parameter| Value                    |
|---------------| -------------------------|
| name          | bpcal_p.G0               |
| field         | bpcal                    |
| previous_cal  | ['bpcal_d.K0']           |
| solint        | int                      |
| gaintype      | G                        |
| calmode       | p                        |
| spw           | *:innerchan              |
| combine       |                          |
| table         | ./calib/inbase_bpcal_p.G0|
| gainfield     | bpcal                    |
| interp        | linear                   |
| spwmap        | []                       |

**bpcal_ap.G1** - Amplitude and phase calibration of bandpass calibrator(s)

|Table parameter| Value                       |
|---------------| --------------------------- |
| name          | bpcal_p.G0                  |
| field         | bpcal                       |
| previous_cal  | ['bpcal_d.K0', 'bpcal_p.G0']|
| solint        | 32s                         |
| gaintype      | G                           |
| calmode       | ap                          |
| spw           | *:innerchan                 |
| combine       |                             |
| table         | ./calib/inbase_bpcal_ap.G1  |
| gainfield     | bpcal                       |
| interp        | linear                      |
| spwmap        | []                          |

**bpcal.B1** - Initial bandpass table

|Table parameter| Value                                      |
|-------------- | ------------------------------------------ |
| name          | bpcal.B0                                   |
| field         | bpcal                                      |
| previous_cal  | ['bpcal_d.K0', 'bpcal_p.G0', 'bpcal_ap.G1']|
| solint        | inf                                        |
| spw           |                                            |
| uvrange       |                                            |
| combine       | field,scan                                 |
| table         | ./calib/inbase_bpcal.B0                    |
| gainfield     | bpcal                                      |
| interp        | nearest,linear                             |
| spwmap        | []                                         |

**Applycal:**

 - On calibrators: ['bpcal.B0']
 - On targets: ['bpcal.B0']


In [22]:
inputs["init_models"] = 0
inputs["bandpass_0"] = 1

run_pipeline(inputs=inputs)

2018-07-09 12:09:11 | INFO | Starting pipeline
2018-07-09 12:09:11 | INFO | Starting pipeline
2018-07-09 12:09:11 | INFO | Starting pipeline
2018-07-09 12:09:11 | INFO | Starting pipeline
2018-07-09 12:09:11 | INFO | Starting pipeline
2018-07-09 12:09:12 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:09:12 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:09:12 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:09:12 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:09:12 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:09:12 | INFO | Pipeline version: v0.7.11
2018-07-09 12:09:12 | INFO | Pipeline version: v0.7.11
2018-07-09 12:09:12 | INFO | Pipeline version: v0.7.11
2018-07-09 12:09:12 | INFO | Pipeline version: v0.7.11
2018-07-09 12:09:12 | INFO | Pipeline version: v0.7.11
2018-07-09 12:09:12 | INFO | Using github branch: jupyter_notebook
2018-07-09 12:09:12 | INFO | Using github b

2018-07-09 12:09:14 | INFO | Previous calibration spwmap: []
2018-07-09 12:09:14 | INFO | Previous calibration interp: []
2018-07-09 12:09:14 | INFO | Previous calibration interp: []
2018-07-09 12:09:14 | INFO | Previous calibration interp: []
2018-07-09 12:09:14 | INFO | Previous calibration interp: []
2018-07-09 12:09:14 | INFO | Previous calibration interp: []
2018-07-09 12:09:14 | INFO | Generating calibration table: ./calib/all_avg_bpcal_d.K0
2018-07-09 12:09:14 | INFO | Generating calibration table: ./calib/all_avg_bpcal_d.K0
2018-07-09 12:09:14 | INFO | Generating calibration table: ./calib/all_avg_bpcal_d.K0
2018-07-09 12:09:14 | INFO | Generating calibration table: ./calib/all_avg_bpcal_d.K0
2018-07-09 12:09:14 | INFO | Generating calibration table: ./calib/all_avg_bpcal_d.K0
2018-07-09 12:09:17 | INFO | caltable bpcal_d.K0 in ./calib/all_avg_bpcal_d.K0
2018-07-09 12:09:17 | INFO | caltable bpcal_d.K0 in ./calib/all_avg_bpcal_d.K0
2018-07-09 12:09:17 | INFO | caltable bpcal_d.

2018-07-09 12:09:39 | INFO | Previous calibration applied: ['bpcal_d.K0', 'bpcal_p.G0']
2018-07-09 12:09:39 | INFO | Previous calibration applied: ['bpcal_d.K0', 'bpcal_p.G0']
2018-07-09 12:09:39 | INFO | Previous calibration applied: ['bpcal_d.K0', 'bpcal_p.G0']
2018-07-09 12:09:39 | INFO | Previous calibration applied: ['bpcal_d.K0', 'bpcal_p.G0']
2018-07-09 12:09:39 | INFO | Previous calibration applied: ['bpcal_d.K0', 'bpcal_p.G0']
2018-07-09 12:09:39 | INFO | Previous calibration gainfield: ['1407+284', '1407+284']
2018-07-09 12:09:39 | INFO | Previous calibration gainfield: ['1407+284', '1407+284']
2018-07-09 12:09:39 | INFO | Previous calibration gainfield: ['1407+284', '1407+284']
2018-07-09 12:09:39 | INFO | Previous calibration gainfield: ['1407+284', '1407+284']
2018-07-09 12:09:39 | INFO | Previous calibration gainfield: ['1407+284', '1407+284']
2018-07-09 12:09:39 | INFO | Previous calibration spwmap: [[0, 0, 0, 0], []]
2018-07-09 12:09:39 | INFO | Previous calibration spw

2018-07-09 12:09:55 | INFO | Bandpass0 BP bpcal.B0: ./calib/all_avg_bpcal.B0
2018-07-09 12:09:55 | INFO | Bandpass0 BP bpcal.B0: ./calib/all_avg_bpcal.B0
2018-07-09 12:09:55 | INFO | Bandpass0 BP bpcal.B0: ./calib/all_avg_bpcal.B0
2018-07-09 12:09:55 | INFO | Bandpass0 BP bpcal.B0: ./calib/all_avg_bpcal.B0
2018-07-09 12:09:58 | INFO | Bandpass0 BP phase plot: ./plots/caltables/all_avg_bpcal.B0_phs.png
2018-07-09 12:09:58 | INFO | Bandpass0 BP phase plot: ./plots/caltables/all_avg_bpcal.B0_phs.png
2018-07-09 12:09:58 | INFO | Bandpass0 BP phase plot: ./plots/caltables/all_avg_bpcal.B0_phs.png
2018-07-09 12:09:58 | INFO | Bandpass0 BP phase plot: ./plots/caltables/all_avg_bpcal.B0_phs.png
2018-07-09 12:09:58 | INFO | Bandpass0 BP phase plot: ./plots/caltables/all_avg_bpcal.B0_phs.png
2018-07-09 12:10:01 | INFO | Bandpass0 BP amplitude plot: ./plots/caltables/all_avg_bpcal.B0_amp.png
2018-07-09 12:10:01 | INFO | Bandpass0 BP amplitude plot: ./plots/caltables/all_avg_bpcal.B0_amp.png
2018-

({'Lo_dropout_scans': '',
  'applycal_all': 0,
  'average_1': 0,
  'bandpass_0': 1,
  'bandpass_1_sp': 0,
  'bpcal': '1407+284',
  'delay': 0,
  'first_images': 0,
  'fits_path': '/home/jupyter/data/',
  'flag_0_aoflagger': 0,
  'flag_1_apriori': 0,
  'flag_2a_manual': 0,
  'flag_2b_manual': 0,
  'flag_3_tfcropBP': 0,
  'flag_4_rflag': 0,
  'flag_statistics': 0,
  'fluxcal': '1331+305',
  'fluxscale': 0,
  'gain_0_p_ap': 0,
  'gain_1_amp_sp': 0,
  'hanning': 0,
  'inbase': 'all_avg',
  'init_models': 0,
  'manual_flags_a': './inputfg_a.flags',
  'manual_flags_b': './all_avg_1.flags',
  'ms2mms': 0,
  'phscals': '1302+5748',
  'plot_corrected': 0,
  'plot_data': 0,
  'ptcal': '0319+415',
  'refant': 'Mk2',
  'restore_flags': 0,
  'run_importfits': 0,
  'save_flags': 0,
  'shift_field_pos': 0,
  'summary_weblog': 0,
  'targets': '1252+5634',
  'weblog': 0},
 {'antennas': ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm'],
  'band': 'C',
  'baselines': ['Mk2&Kn',
   'Mk2&De',
   'Mk2&Pi',
   'Mk2&Da',

### 4.2.5 flag_3_tfcropBP

This task needs the data to be bandpass corrected. So first of all, it will apply the table ['bpcal.B0'] to all sources.

CASA `flagdata`

|Parameter      | Value  |
|-------------- | ------ |
| mode          | tfcrop |
| correlation   | ABS_ALL|
| ntime         | 90min  |
| combinescans  | True   |
| datacolumn    | DATA   |
| winsize       | 3      |
| timecutoff    | 3.6    |
| freqcutoff    | 3.6    |
| maxnpieces    | 2      |
| usewindowstats| sum    |
| halfwin       | 3      |
| extendflags   | True   |
| action        | apply  |



In [23]:
inputs["bandpass_0"] = 0
inputs["flag_3_tfcropBP"] = 1

run_pipeline(inputs=inputs)

2018-07-09 12:10:01 | INFO | Starting pipeline
2018-07-09 12:10:01 | INFO | Starting pipeline
2018-07-09 12:10:01 | INFO | Starting pipeline
2018-07-09 12:10:01 | INFO | Starting pipeline
2018-07-09 12:10:01 | INFO | Starting pipeline
2018-07-09 12:10:01 | INFO | Starting pipeline
2018-07-09 12:10:01 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:10:01 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:10:01 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:10:01 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:10:01 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:10:01 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:10:02 | INFO | Pipeline version: v0.7.11
2018-07-09 12:10:02 | INFO | Pipeline version: v0.7.11
2018-07-09 12:10:02 | INFO | Pipeline version: v0.7.11
2018-07-09 12:10:02 | INFO | Pipeline version: v0.7.11
2018-07-09 12:10:02 | INFO | Pipeline vers

2018-07-09 12:10:04 | INFO | Previous calibration applied: ['bpcal.B0']
2018-07-09 12:10:04 | INFO | Previous calibration applied: ['bpcal.B0']
2018-07-09 12:10:04 | INFO | Previous calibration gainfield: ['1407+284']
2018-07-09 12:10:04 | INFO | Previous calibration gainfield: ['1407+284']
2018-07-09 12:10:04 | INFO | Previous calibration gainfield: ['1407+284']
2018-07-09 12:10:04 | INFO | Previous calibration gainfield: ['1407+284']
2018-07-09 12:10:04 | INFO | Previous calibration gainfield: ['1407+284']
2018-07-09 12:10:04 | INFO | Previous calibration gainfield: ['1407+284']
2018-07-09 12:10:04 | INFO | Previous calibration spwmap: [[]]
2018-07-09 12:10:04 | INFO | Previous calibration spwmap: [[]]
2018-07-09 12:10:04 | INFO | Previous calibration spwmap: [[]]
2018-07-09 12:10:04 | INFO | Previous calibration spwmap: [[]]
2018-07-09 12:10:04 | INFO | Previous calibration spwmap: [[]]
2018-07-09 12:10:04 | INFO | Previous calibration spwmap: [[]]
2018-07-09 12:10:04 | INFO | Previ

2018-07-09 12:11:40 | INFO | New flags applied: flagdata3_tfcropBP
2018-07-09 12:11:40 | INFO | Current flags applied: ['flagdata_manual', 'flagdata_manual', 'flagdata_manual', 'flagdata3_tfcropBP', 'flagdata_manual', 'flagdata3_tfcropBP']
2018-07-09 12:11:40 | INFO | Current flags applied: ['flagdata_manual', 'flagdata_manual', 'flagdata_manual', 'flagdata3_tfcropBP', 'flagdata_manual', 'flagdata3_tfcropBP']
2018-07-09 12:11:40 | INFO | Current flags applied: ['flagdata_manual', 'flagdata_manual', 'flagdata_manual', 'flagdata3_tfcropBP', 'flagdata_manual', 'flagdata3_tfcropBP']
2018-07-09 12:11:40 | INFO | Current flags applied: ['flagdata_manual', 'flagdata_manual', 'flagdata_manual', 'flagdata3_tfcropBP', 'flagdata_manual', 'flagdata3_tfcropBP']
2018-07-09 12:11:40 | INFO | Current flags applied: ['flagdata_manual', 'flagdata_manual', 'flagdata_manual', 'flagdata3_tfcropBP', 'flagdata_manual', 'flagdata3_tfcropBP']
2018-07-09 12:11:40 | INFO | Current flags applied: ['flagdata_manua

({'Lo_dropout_scans': '',
  'applycal_all': 0,
  'average_1': 0,
  'bandpass_0': 0,
  'bandpass_1_sp': 0,
  'bpcal': '1407+284',
  'delay': 0,
  'first_images': 0,
  'fits_path': '/home/jupyter/data/',
  'flag_0_aoflagger': 0,
  'flag_1_apriori': 0,
  'flag_2a_manual': 0,
  'flag_2b_manual': 0,
  'flag_3_tfcropBP': 1,
  'flag_4_rflag': 0,
  'flag_statistics': 0,
  'fluxcal': '1331+305',
  'fluxscale': 0,
  'gain_0_p_ap': 0,
  'gain_1_amp_sp': 0,
  'hanning': 0,
  'inbase': 'all_avg',
  'init_models': 0,
  'manual_flags_a': './inputfg_a.flags',
  'manual_flags_b': './all_avg_1.flags',
  'ms2mms': 0,
  'phscals': '1302+5748',
  'plot_corrected': 0,
  'plot_data': 0,
  'ptcal': '0319+415',
  'refant': 'Mk2',
  'restore_flags': 0,
  'run_importfits': 0,
  'save_flags': 0,
  'shift_field_pos': 0,
  'summary_weblog': 0,
  'targets': '1252+5634',
  'weblog': 0},
 {'antennas': ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm'],
  'band': 'C',
  'baselines': ['Mk2&Kn',
   'Mk2&De',
   'Mk2&Pi',
   'Mk2&Da',

### 4.2.6 delay

**bpcal_d.K0** - Delay calibration of all calibrators.

|Table parameter| Value                  |
|---------------| -----------------------|
| name          | delay.K1               |
| field         | calsources             |
| previous_cal  | ['bpcal.B0']           |
| solint        | 300s                   |
| gaintype      | K                      |
| calmode       | p                      |
| spw           | *:innerchan            |
| combine       | spw                    |
| table         | ./calib/inbase_delay.K1|
| gainfield     | calsources             |
| interp        | linear                 |
| spwmap        | [0]*num_spw            |

**Applycal:**

 - On calibrators: ['bpcal.B0','delay.K1']
 - On targets: ['bpcal.B0','delay.K1']


In [24]:
inputs["flag_3_tfcropBP"] = 0
inputs["delay"] = 1

run_pipeline(inputs=inputs)

2018-07-09 12:11:40 | INFO | Starting pipeline
2018-07-09 12:11:40 | INFO | Starting pipeline
2018-07-09 12:11:40 | INFO | Starting pipeline
2018-07-09 12:11:40 | INFO | Starting pipeline
2018-07-09 12:11:40 | INFO | Starting pipeline
2018-07-09 12:11:40 | INFO | Starting pipeline
2018-07-09 12:11:40 | INFO | Starting pipeline
2018-07-09 12:11:40 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:11:40 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:11:40 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:11:40 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:11:40 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:11:40 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:11:40 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:11:40 | INFO | Pipeline version: v0.7.11
2018-07-09 12:11:40 | INFO | Pipeline version: v0.7.11
2018-07-09 12:11:40 | INFO | 

2018-07-09 12:11:43 | INFO | Start solve_delays
2018-07-09 12:11:43 | INFO | Deleted: ./calib/all_avg_delay.K1
2018-07-09 12:11:43 | INFO | Deleted: ./calib/all_avg_delay.K1
2018-07-09 12:11:43 | INFO | Deleted: ./calib/all_avg_delay.K1
2018-07-09 12:11:43 | INFO | Deleted: ./calib/all_avg_delay.K1
2018-07-09 12:11:43 | INFO | Deleted: ./calib/all_avg_delay.K1
2018-07-09 12:11:43 | INFO | Deleted: ./calib/all_avg_delay.K1
2018-07-09 12:11:43 | INFO | Deleted: ./calib/all_avg_delay.K1
2018-07-09 12:11:43 | INFO | Running gaincal to generate: delay.K1
2018-07-09 12:11:43 | INFO | Running gaincal to generate: delay.K1
2018-07-09 12:11:43 | INFO | Running gaincal to generate: delay.K1
2018-07-09 12:11:43 | INFO | Running gaincal to generate: delay.K1
2018-07-09 12:11:43 | INFO | Running gaincal to generate: delay.K1
2018-07-09 12:11:43 | INFO | Running gaincal to generate: delay.K1
2018-07-09 12:11:43 | INFO | Running gaincal to generate: delay.K1
2018-07-09 12:11:43 | INFO | Field(s) = 03

({'Lo_dropout_scans': '',
  'applycal_all': 0,
  'average_1': 0,
  'bandpass_0': 0,
  'bandpass_1_sp': 0,
  'bpcal': '1407+284',
  'delay': 1,
  'first_images': 0,
  'fits_path': '/home/jupyter/data/',
  'flag_0_aoflagger': 0,
  'flag_1_apriori': 0,
  'flag_2a_manual': 0,
  'flag_2b_manual': 0,
  'flag_3_tfcropBP': 0,
  'flag_4_rflag': 0,
  'flag_statistics': 0,
  'fluxcal': '1331+305',
  'fluxscale': 0,
  'gain_0_p_ap': 0,
  'gain_1_amp_sp': 0,
  'hanning': 0,
  'inbase': 'all_avg',
  'init_models': 0,
  'manual_flags_a': './inputfg_a.flags',
  'manual_flags_b': './all_avg_1.flags',
  'ms2mms': 0,
  'phscals': '1302+5748',
  'plot_corrected': 0,
  'plot_data': 0,
  'ptcal': '0319+415',
  'refant': 'Mk2',
  'restore_flags': 0,
  'run_importfits': 0,
  'save_flags': 0,
  'shift_field_pos': 0,
  'summary_weblog': 0,
  'targets': '1252+5634',
  'weblog': 0},
 {'antennas': ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm'],
  'band': 'C',
  'baselines': ['Mk2&Kn',
   'Mk2&De',
   'Mk2&Pi',
   'Mk2&Da',

### 4.2.7 gain_0_p_ap

**allcal_p.G0** - Phase calibration of all calibrators.

|Table parameter| Value                     |
|---------------| --------------------------|
| name          | allcal_p.G0               |
| field         | calsources                |
| previous_cal  | ['delay.K1', 'bpcal.B0']  |
| solint        | 16s                       |
| gaintype      | G                         |
| calmode       | p                         |
| spw           | *:innerchan               |
| combine       |                           |
| table         | ./calib/inbase_allcal_p.G0|
| gainfield     | calsources                |
| interp        | linear                    |
| spwmap        | []                        |

**allcal_p_jitter.G0** - Short-interval phase calibration of all calibrators.

|Table parameter| Value                            |
|---------------| ---------------------------------|
| name          | allcal_p_jitter.G0               |
| field         | calsources                       |
| previous_cal  | ['delay.K1', 'bpcal.B0']         |
| solint        | 2s                               |
| gaintype      | G                                |
| calmode       | p                                |
| spw           | *:innerchan                      |
| combine       | spw                              |
| table         | ./calib/inbase_allcal_p_jitter.G0|
| gainfield     | calsources                       |
| interp        | linear                           |
| spwmap        | [0]*nchan                        |


**allcal_ap.G1** - Amplitude and phase calibration of all calibrators.

|Table parameter| Value                                                        |
|---------------| ------------------------------------------------------------ |
| name          | allcal_ap.G1                                                 |
| field         | calsources                                                   |
| previous_cal  | ['delay.K1', 'bpcal.B0', 'allcal_p.G0', 'allcal_p_jitter.G0']|
| solint        | 32s                                                          |
| gaintype      | G                                                            |
| calmode       | ap                                                           |
| spw           | *:innerchan                                                  |
| combine       |                                                              |
| table         | ./calib/inbase_allcal_ap.G1                                  |
| gainfield     | calsources                                                   |
| interp        | linear                                                       |
| spwmap        | []                                                           |


**phscal_p_scan.G2** - Scan-averaged phase calibration of all phase reference calibrators.

|Table parameter| Value                          |
|---------------| -------------------------------|
| name          | phscal_p_scan.G2               |
| field         | calsources                     |
| previous_cal  | ['delay.K1', 'bpcal.B0']       |
| solint        | inf                            |
| gaintype      | G                              |
| calmode       | p                              |
| spw           | *:innerchan                    |
| combine       |                                |
| table         | ./calib/inbase_phscal_p_scan.G2|
| gainfield     | calsources                     |
| interp        | linear                         |
| spwmap        | []                             |



**Applycal:**

 - On calibrators: ['delay.K1','allcal_p.G0', 'allcal_p_jitter.G0', 'allcal_ap.G1','bpcal.B0']
 - On targets: ['delay.K1','phscal_p_scan.G2','allcal_ap.G1','bpcal.B0']


In [25]:
inputs["delay"] = 0
inputs["gain_0_p_ap"] = 1

run_pipeline(inputs=inputs)

2018-07-09 12:12:10 | INFO | Starting pipeline
2018-07-09 12:12:10 | INFO | Starting pipeline
2018-07-09 12:12:10 | INFO | Starting pipeline
2018-07-09 12:12:10 | INFO | Starting pipeline
2018-07-09 12:12:10 | INFO | Starting pipeline
2018-07-09 12:12:10 | INFO | Starting pipeline
2018-07-09 12:12:10 | INFO | Starting pipeline
2018-07-09 12:12:10 | INFO | Starting pipeline
2018-07-09 12:12:10 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:12:10 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:12:10 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:12:10 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:12:10 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:12:10 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:12:10 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:12:10 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:12

2018-07-09 12:12:12 | INFO | Refant: Mk2
2018-07-09 12:12:12 | INFO | Refant: Mk2
2018-07-09 12:12:12 | INFO | Refant: Mk2
2018-07-09 12:12:12 | INFO | Loaded previous calibration tables from: ./calib/caltables.pkl
2018-07-09 12:12:12 | INFO | Loaded previous calibration tables from: ./calib/caltables.pkl
2018-07-09 12:12:12 | INFO | Loaded previous calibration tables from: ./calib/caltables.pkl
2018-07-09 12:12:12 | INFO | Loaded previous calibration tables from: ./calib/caltables.pkl
2018-07-09 12:12:12 | INFO | Loaded previous calibration tables from: ./calib/caltables.pkl
2018-07-09 12:12:12 | INFO | Loaded previous calibration tables from: ./calib/caltables.pkl
2018-07-09 12:12:12 | INFO | Loaded previous calibration tables from: ./calib/caltables.pkl
2018-07-09 12:12:12 | INFO | Loaded previous calibration tables from: ./calib/caltables.pkl
2018-07-09 12:12:12 | INFO | Start initial_gaincal
2018-07-09 12:12:12 | INFO | Start initial_gaincal
2018-07-09 12:12:12 | INFO | Start init

2018-07-09 12:12:52 | INFO | Gain phase calibration allcal_p.G0: ./calib/all_avg_allcal_p.G0
2018-07-09 12:12:52 | INFO | Gain phase calibration allcal_p.G0: ./calib/all_avg_allcal_p.G0
2018-07-09 12:13:00 | INFO | Gain phase calibration plot: ./plots/caltables/all_avg_allcal_p.G0_phs.png
2018-07-09 12:13:00 | INFO | Gain phase calibration plot: ./plots/caltables/all_avg_allcal_p.G0_phs.png
2018-07-09 12:13:00 | INFO | Gain phase calibration plot: ./plots/caltables/all_avg_allcal_p.G0_phs.png
2018-07-09 12:13:00 | INFO | Gain phase calibration plot: ./plots/caltables/all_avg_allcal_p.G0_phs.png
2018-07-09 12:13:00 | INFO | Gain phase calibration plot: ./plots/caltables/all_avg_allcal_p.G0_phs.png
2018-07-09 12:13:00 | INFO | Gain phase calibration plot: ./plots/caltables/all_avg_allcal_p.G0_phs.png
2018-07-09 12:13:00 | INFO | Gain phase calibration plot: ./plots/caltables/all_avg_allcal_p.G0_phs.png
2018-07-09 12:13:00 | INFO | Gain phase calibration plot: ./plots/caltables/all_avg_al

2018-07-09 12:14:23 | INFO | Gain phase calibration allcal_p_jitter.G0: ./calib/all_avg_allcal_p_jitter.G0
2018-07-09 12:14:23 | INFO | Gain phase calibration allcal_p_jitter.G0: ./calib/all_avg_allcal_p_jitter.G0
2018-07-09 12:14:23 | INFO | Gain phase calibration allcal_p_jitter.G0: ./calib/all_avg_allcal_p_jitter.G0
2018-07-09 12:14:23 | INFO | Gain phase calibration allcal_p_jitter.G0: ./calib/all_avg_allcal_p_jitter.G0
2018-07-09 12:14:23 | INFO | Gain phase calibration allcal_p_jitter.G0: ./calib/all_avg_allcal_p_jitter.G0
2018-07-09 12:14:23 | INFO | Gain phase calibration allcal_p_jitter.G0: ./calib/all_avg_allcal_p_jitter.G0
2018-07-09 12:14:23 | INFO | Deleted: ./calib/all_avg_allcal_ap.G1
2018-07-09 12:14:23 | INFO | Deleted: ./calib/all_avg_allcal_ap.G1
2018-07-09 12:14:23 | INFO | Deleted: ./calib/all_avg_allcal_ap.G1
2018-07-09 12:14:23 | INFO | Deleted: ./calib/all_avg_allcal_ap.G1
2018-07-09 12:14:23 | INFO | Deleted: ./calib/all_avg_allcal_ap.G1
2018-07-09 12:14:23 | I

2018-07-09 12:14:57 | INFO | Gain amplitude calibration allcal_ap.G1: ./calib/all_avg_allcal_ap.G1
2018-07-09 12:14:57 | INFO | Gain amplitude calibration allcal_ap.G1: ./calib/all_avg_allcal_ap.G1
2018-07-09 12:14:57 | INFO | Gain amplitude calibration allcal_ap.G1: ./calib/all_avg_allcal_ap.G1
2018-07-09 12:15:05 | INFO | Bandpass0 phase calibration plot: ./plots/caltables/all_avg_allcal_p_jitter.G0_phs.png
2018-07-09 12:15:05 | INFO | Bandpass0 phase calibration plot: ./plots/caltables/all_avg_allcal_p_jitter.G0_phs.png
2018-07-09 12:15:05 | INFO | Bandpass0 phase calibration plot: ./plots/caltables/all_avg_allcal_p_jitter.G0_phs.png
2018-07-09 12:15:05 | INFO | Bandpass0 phase calibration plot: ./plots/caltables/all_avg_allcal_p_jitter.G0_phs.png
2018-07-09 12:15:05 | INFO | Bandpass0 phase calibration plot: ./plots/caltables/all_avg_allcal_p_jitter.G0_phs.png
2018-07-09 12:15:05 | INFO | Bandpass0 phase calibration plot: ./plots/caltables/all_avg_allcal_p_jitter.G0_phs.png
2018-07

2018-07-09 12:15:22 | INFO | Gain phase calibration phscal_p_scan.G2: ./calib/all_avg_phscal_p_scan.G2
2018-07-09 12:15:22 | INFO | Gain phase calibration phscal_p_scan.G2: ./calib/all_avg_phscal_p_scan.G2
2018-07-09 12:15:26 | INFO | Gain phase calibration plot: ./plots/caltables/all_avg_phscal_p_scan.G2_phs.png
2018-07-09 12:15:26 | INFO | Gain phase calibration plot: ./plots/caltables/all_avg_phscal_p_scan.G2_phs.png
2018-07-09 12:15:26 | INFO | Gain phase calibration plot: ./plots/caltables/all_avg_phscal_p_scan.G2_phs.png
2018-07-09 12:15:26 | INFO | Gain phase calibration plot: ./plots/caltables/all_avg_phscal_p_scan.G2_phs.png
2018-07-09 12:15:26 | INFO | Gain phase calibration plot: ./plots/caltables/all_avg_phscal_p_scan.G2_phs.png
2018-07-09 12:15:26 | INFO | Gain phase calibration plot: ./plots/caltables/all_avg_phscal_p_scan.G2_phs.png
2018-07-09 12:15:26 | INFO | Gain phase calibration plot: ./plots/caltables/all_avg_phscal_p_scan.G2_phs.png
2018-07-09 12:15:26 | INFO | Ga

({'Lo_dropout_scans': '',
  'applycal_all': 0,
  'average_1': 0,
  'bandpass_0': 0,
  'bandpass_1_sp': 0,
  'bpcal': '1407+284',
  'delay': 0,
  'first_images': 0,
  'fits_path': '/home/jupyter/data/',
  'flag_0_aoflagger': 0,
  'flag_1_apriori': 0,
  'flag_2a_manual': 0,
  'flag_2b_manual': 0,
  'flag_3_tfcropBP': 0,
  'flag_4_rflag': 0,
  'flag_statistics': 0,
  'fluxcal': '1331+305',
  'fluxscale': 0,
  'gain_0_p_ap': 1,
  'gain_1_amp_sp': 0,
  'hanning': 0,
  'inbase': 'all_avg',
  'init_models': 0,
  'manual_flags_a': './inputfg_a.flags',
  'manual_flags_b': './all_avg_1.flags',
  'ms2mms': 0,
  'phscals': '1302+5748',
  'plot_corrected': 0,
  'plot_data': 0,
  'ptcal': '0319+415',
  'refant': 'Mk2',
  'restore_flags': 0,
  'run_importfits': 0,
  'save_flags': 0,
  'shift_field_pos': 0,
  'summary_weblog': 0,
  'targets': '1252+5634',
  'weblog': 0},
 {'antennas': ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm'],
  'band': 'C',
  'baselines': ['Mk2&Kn',
   'Mk2&De',
   'Mk2&Pi',
   'Mk2&Da',

### 4.2.8 fluxscale

Runs CASA `fluxscale` to bootstrap the flux density scale of 1331+305 using its model, and forward the corrections to all other sources, updating their model column. Also, a corrected _fluxscale table is derived from the previous amplitude calibration.

CASA `fluxscale`

|Parameter      | Value                                     |
|-------------- | ----------------------------------------- |
| reference     | fluxcal                                   |
| transfer      | calibrators (except fluxcal)              |
| antenna       | anten_for_flux*                           |
| caltable      | allcal_ap.G1                              |
| fluxtable     | allcal_ap.G1_fluxscaled                   |
| listfile      | ./calib/allcal_ap.G1_fluxscaled_fluxes.txt|
|               |                                           |

*anten_for_flux: selection of antennas to use for amplitude scale. Will try to remove Lo and De from the fluxscale determination, but only if there are enough antennas to have at least 4 antennas.


Then, the pipeline will run the script `dfluxpy` to find the correction factor eMfactor. This is a correction factor for the flux density of 1331+305 (3C286) needed because the source is slightly resolved by the shortest baseline of e-MERLIN. The task will check the shortest baseline and the observation frequency, and scale the results accordingly. The values reported by the logger in eMCP.log are already corrected by this factor. The file allcal_ap.G1_fluxscaled_fluxes.txt is not corrected by this factor, but a warning note is included in the file.

Finally, CASA `setjy` is run for each calibrator source (except the fluxcal) to include the new flux density into the model column for each spw.


**Applycal:**

 - On calibrators: ['delay.K1','allcal_p.G0','allcal_p_jitter.G0','allcal_ap.G1_fluxscaled','bpcal.B0']
 - On targets: ['delay.K1','phscal_p_scan.G2','allcal_ap.G1_fluxscaled','bpcal.B0']


In [26]:
inputs["gain_0_p_ap"] = 0
inputs["fluxscale"] = 1

run_pipeline(inputs=inputs)

2018-07-09 12:15:26 | INFO | Starting pipeline
2018-07-09 12:15:26 | INFO | Starting pipeline
2018-07-09 12:15:26 | INFO | Starting pipeline
2018-07-09 12:15:26 | INFO | Starting pipeline
2018-07-09 12:15:26 | INFO | Starting pipeline
2018-07-09 12:15:26 | INFO | Starting pipeline
2018-07-09 12:15:26 | INFO | Starting pipeline
2018-07-09 12:15:26 | INFO | Starting pipeline
2018-07-09 12:15:26 | INFO | Starting pipeline
2018-07-09 12:15:26 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:15:26 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:15:26 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:15:26 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:15:26 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:15:26 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:15:26 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:15:26 | INFO | Running pipeline

2018-07-09 12:15:27 | INFO | Antennas in MS all_avg.ms: ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm']
2018-07-09 12:15:27 | INFO | Antennas in MS all_avg.ms: ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm']
2018-07-09 12:15:27 | INFO | Antennas in MS all_avg.ms: ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm']
2018-07-09 12:15:27 | INFO | Antennas in MS all_avg.ms: ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm']
2018-07-09 12:15:27 | INFO | Antennas in MS all_avg.ms: ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm']
2018-07-09 12:15:27 | INFO | Antennas in MS all_avg.ms: ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm']
2018-07-09 12:15:27 | INFO | Antennas in MS all_avg.ms: ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm']
2018-07-09 12:15:27 | INFO | Antennas in MS all_avg.ms: ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm']
2018-07-09 12:15:28 | INFO | Refant: Mk2
2018-07-09 12:15:28 | INFO | Refant: Mk2
2018-07-09 12:15:28 | INFO | Refant: Mk2
2018-07-09 12:15:28 | INFO | Refant: Mk2
2018-07-09 12:15:28 | INFO | Refant: Mk2
2018-07-09 12:15:28 | INFO | Refant: Mk2
20

2018-07-09 12:15:33 | INFO | Central frequency of the MS: 5072.0 MHz
2018-07-09 12:15:33 | INFO | Central frequency of the MS: 5072.0 MHz
2018-07-09 12:15:33 | INFO | Central frequency of the MS: 5072.0 MHz
2018-07-09 12:15:33 | INFO | Central frequency of the MS: 5072.0 MHz
2018-07-09 12:15:33 | INFO | Central frequency of the MS: 5072.0 MHz
2018-07-09 12:15:33 | INFO | eMfactor: 0.9651
2018-07-09 12:15:33 | INFO | eMfactor: 0.9651
2018-07-09 12:15:33 | INFO | eMfactor: 0.9651
2018-07-09 12:15:33 | INFO | eMfactor: 0.9651
2018-07-09 12:15:33 | INFO | eMfactor: 0.9651
2018-07-09 12:15:33 | INFO | eMfactor: 0.9651
2018-07-09 12:15:33 | INFO | eMfactor: 0.9651
2018-07-09 12:15:33 | INFO | eMfactor: 0.9651
2018-07-09 12:15:33 | INFO | eMfactor: 0.9651
2018-07-09 12:15:33 | INFO | Spectrum for 1302+5748: Flux density = 1.074 +/- 0.022, spidx = 0.28+/- 0.70
2018-07-09 12:15:33 | INFO | Spectrum for 1302+5748: Flux density = 1.074 +/- 0.022, spidx = 0.28+/- 0.70
2018-07-09 12:15:33 | INFO | 

({'Lo_dropout_scans': '',
  'applycal_all': 0,
  'average_1': 0,
  'bandpass_0': 0,
  'bandpass_1_sp': 0,
  'bpcal': '1407+284',
  'delay': 0,
  'first_images': 0,
  'fits_path': '/home/jupyter/data/',
  'flag_0_aoflagger': 0,
  'flag_1_apriori': 0,
  'flag_2a_manual': 0,
  'flag_2b_manual': 0,
  'flag_3_tfcropBP': 0,
  'flag_4_rflag': 0,
  'flag_statistics': 0,
  'fluxcal': '1331+305',
  'fluxscale': 1,
  'gain_0_p_ap': 0,
  'gain_1_amp_sp': 0,
  'hanning': 0,
  'inbase': 'all_avg',
  'init_models': 0,
  'manual_flags_a': './inputfg_a.flags',
  'manual_flags_b': './all_avg_1.flags',
  'ms2mms': 0,
  'phscals': '1302+5748',
  'plot_corrected': 0,
  'plot_data': 0,
  'ptcal': '0319+415',
  'refant': 'Mk2',
  'restore_flags': 0,
  'run_importfits': 0,
  'save_flags': 0,
  'shift_field_pos': 0,
  'summary_weblog': 0,
  'targets': '1252+5634',
  'weblog': 0},
 {'antennas': ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm'],
  'band': 'C',
  'baselines': ['Mk2&Kn',
   'Mk2&De',
   'Mk2&Pi',
   'Mk2&Da',

### 4.2.9 bandpass_1_sp

**bpcal_sp.B1**

Recalculate the BP table. Now the model columns contain the actual flux density of the calibrators, so the BP table will include spectral index information. I use `allcal_ap.G1_fluxscaled` table just because I want to have the right flux scale if applycal is selected for this step.

|Table parameter| Value                                                                    |
|-------------- | ------------------------------------------------------------------------ |
| name          | bpcal_sp.B1                                                              |
| field         | bpcal                                                                    |
| previous_cal  | ['delay.K1','allcal_p.G0','allcal_p_jitter.G0','allcal_ap.G1_fluxscaled']|
| solint        | inf                                                                      |
| spw           |                                                                          |
| uvrange       |                                                                          |
| combine       | field,scan                                                               |
| solnorm       | False                                                                    |
| table         | ./calib/inbase_bpcal_sp.B1                                               |
| gainfield     | bpcal                                                                    |
| interp        | nearest,linear                                                           |
| spwmap        | []                                                                       |

**Applycal:**

 - On calibrators: ['delay.K1','allcal_p.G0','allcal_p_jitter.G0','allcal_ap.G1_fluxscaled','bpcal_sp.B1']
 - On targets: ['delay.K1','phscal_p_scan.G2','allcal_ap.G1_fluxscaled','bpcal_sp.B1']



In [27]:
inputs["fluxscale"] = 0
inputs["bandpass_1_sp"] = 1

run_pipeline(inputs=inputs)

2018-07-09 12:16:00 | INFO | Starting pipeline
2018-07-09 12:16:00 | INFO | Starting pipeline
2018-07-09 12:16:00 | INFO | Starting pipeline
2018-07-09 12:16:00 | INFO | Starting pipeline
2018-07-09 12:16:00 | INFO | Starting pipeline
2018-07-09 12:16:00 | INFO | Starting pipeline
2018-07-09 12:16:00 | INFO | Starting pipeline
2018-07-09 12:16:00 | INFO | Starting pipeline
2018-07-09 12:16:00 | INFO | Starting pipeline
2018-07-09 12:16:00 | INFO | Starting pipeline
2018-07-09 12:16:00 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:16:00 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:16:00 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:16:00 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:16:00 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:16:00 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:16:00 | INFO | Running pipeline from: /home/jupyter/pipeline

2018-07-09 12:16:01 | INFO | Sources in MS all_avg.ms: 1302+5748,1252+5634,1331+305,1407+284,0319+415
2018-07-09 12:16:01 | INFO | Sources in MS all_avg.ms: 1302+5748,1252+5634,1331+305,1407+284,0319+415
2018-07-09 12:16:01 | INFO | Sources in MS all_avg.ms: 1302+5748,1252+5634,1331+305,1407+284,0319+415
2018-07-09 12:16:01 | INFO | Sources in MS all_avg.ms: 1302+5748,1252+5634,1331+305,1407+284,0319+415
2018-07-09 12:16:01 | INFO | Sources in MS all_avg.ms: 1302+5748,1252+5634,1331+305,1407+284,0319+415
2018-07-09 12:16:01 | INFO | Sources in MS all_avg.ms: 1302+5748,1252+5634,1331+305,1407+284,0319+415
2018-07-09 12:16:01 | INFO | Sources in MS all_avg.ms: 1302+5748,1252+5634,1331+305,1407+284,0319+415
2018-07-09 12:16:01 | INFO | Sources in MS all_avg.ms: 1302+5748,1252+5634,1331+305,1407+284,0319+415
2018-07-09 12:16:01 | INFO | Antennas in MS all_avg.ms: ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm']
2018-07-09 12:16:01 | INFO | Antennas in MS all_avg.ms: ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'C

2018-07-09 12:16:03 | INFO | Previous calibration gainfield: ['', '', '', '']
2018-07-09 12:16:03 | INFO | Previous calibration gainfield: ['', '', '', '']
2018-07-09 12:16:03 | INFO | Previous calibration gainfield: ['', '', '', '']
2018-07-09 12:16:03 | INFO | Previous calibration gainfield: ['', '', '', '']
2018-07-09 12:16:03 | INFO | Previous calibration spwmap: [[0, 0, 0, 0], [], [0, 0, 0, 0], []]
2018-07-09 12:16:03 | INFO | Previous calibration spwmap: [[0, 0, 0, 0], [], [0, 0, 0, 0], []]
2018-07-09 12:16:03 | INFO | Previous calibration spwmap: [[0, 0, 0, 0], [], [0, 0, 0, 0], []]
2018-07-09 12:16:03 | INFO | Previous calibration spwmap: [[0, 0, 0, 0], [], [0, 0, 0, 0], []]
2018-07-09 12:16:03 | INFO | Previous calibration spwmap: [[0, 0, 0, 0], [], [0, 0, 0, 0], []]
2018-07-09 12:16:03 | INFO | Previous calibration spwmap: [[0, 0, 0, 0], [], [0, 0, 0, 0], []]
2018-07-09 12:16:03 | INFO | Previous calibration spwmap: [[0, 0, 0, 0], [], [0, 0, 0, 0], []]
2018-07-09 12:16:03 | I

({'Lo_dropout_scans': '',
  'applycal_all': 0,
  'average_1': 0,
  'bandpass_0': 0,
  'bandpass_1_sp': 1,
  'bpcal': '1407+284',
  'delay': 0,
  'first_images': 0,
  'fits_path': '/home/jupyter/data/',
  'flag_0_aoflagger': 0,
  'flag_1_apriori': 0,
  'flag_2a_manual': 0,
  'flag_2b_manual': 0,
  'flag_3_tfcropBP': 0,
  'flag_4_rflag': 0,
  'flag_statistics': 0,
  'fluxcal': '1331+305',
  'fluxscale': 0,
  'gain_0_p_ap': 0,
  'gain_1_amp_sp': 0,
  'hanning': 0,
  'inbase': 'all_avg',
  'init_models': 0,
  'manual_flags_a': './inputfg_a.flags',
  'manual_flags_b': './all_avg_1.flags',
  'ms2mms': 0,
  'phscals': '1302+5748',
  'plot_corrected': 0,
  'plot_data': 0,
  'ptcal': '0319+415',
  'refant': 'Mk2',
  'restore_flags': 0,
  'run_importfits': 0,
  'save_flags': 0,
  'shift_field_pos': 0,
  'summary_weblog': 0,
  'targets': '1252+5634',
  'weblog': 0},
 {'antennas': ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm'],
  'band': 'C',
  'baselines': ['Mk2&Kn',
   'Mk2&De',
   'Mk2&Pi',
   'Mk2&Da',

### 4.2.10 gain_1_amp_sp

**allcal_ap.G3**


|Table parameter| Value                                                        |
|---------------| ------------------------------------------------------------ |
| name          | allcal_ap.G3                                                 |
| field         | calsources                                                   |
| previous_cal  | ['delay.K1','allcal_p.G0','allcal_p_jitter.G0','bpcal_sp.B1']|
| solint        | 32s                                                          |
| gaintype      | G                                                            |
| calmode       | ap                                                           |
| spw           | *:innerchan                                                  |
| combine       |                                                              |
| table         | ./calib/inbase_allcal_ap.G3                                  |
| gainfield     | calsources                                                   |
| interp        | linear                                                       |
| spwmap        | []                                                           |

**allcal_ap_scan.G3**


|Table parameter| Value                                                        |
|---------------| ------------------------------------------------------------ |
| name          | allcal_ap_scan.G3                                            |
| field         | phscals                                                      |
| previous_cal  | ['delay.K1','allcal_p.G0','allcal_p_jitter.G0','bpcal_sp.B1']|
| solint        | inf                                                          |
| gaintype      | G                                                            |
| calmode       | ap                                                           |
| spw           | *:innerchan                                                  |
| combine       |                                                              |
| table         | ./calib/inbase_allcal_ap_scan.G3                             |
| gainfield     | phscals                                                      |
| interp        | linear                                                       |
| spwmap        | []                                                           |



**Applycal:**

 - On calibrators: ['delay.K1','bpcal_sp.B1','allcal_p.G0','allcal_p_jitter.G0','allcal_ap.G3']
 - On targets: ['delay.K1','bpcal_sp.B1','phscal_p_scan.G2','allcal_ap_scan.G3']


In [28]:
inputs["bandpass_1_sp"] = 0
inputs["gain_1_amp_sp"] = 1

run_pipeline(inputs=inputs)

2018-07-09 12:16:15 | INFO | Starting pipeline
2018-07-09 12:16:15 | INFO | Starting pipeline
2018-07-09 12:16:15 | INFO | Starting pipeline
2018-07-09 12:16:15 | INFO | Starting pipeline
2018-07-09 12:16:15 | INFO | Starting pipeline
2018-07-09 12:16:15 | INFO | Starting pipeline
2018-07-09 12:16:15 | INFO | Starting pipeline
2018-07-09 12:16:15 | INFO | Starting pipeline
2018-07-09 12:16:15 | INFO | Starting pipeline
2018-07-09 12:16:15 | INFO | Starting pipeline
2018-07-09 12:16:15 | INFO | Starting pipeline
2018-07-09 12:16:15 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:16:15 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:16:15 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:16:15 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:16:15 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:16:15 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:16:15 | INFO 

2018-07-09 12:16:15 | INFO | Bandpass:  1407+284
2018-07-09 12:16:15 | INFO | Bandpass:  1407+284
2018-07-09 12:16:15 | INFO | Pointcal:  0319+415
2018-07-09 12:16:15 | INFO | Pointcal:  0319+415
2018-07-09 12:16:15 | INFO | Pointcal:  0319+415
2018-07-09 12:16:15 | INFO | Pointcal:  0319+415
2018-07-09 12:16:15 | INFO | Pointcal:  0319+415
2018-07-09 12:16:15 | INFO | Pointcal:  0319+415
2018-07-09 12:16:15 | INFO | Pointcal:  0319+415
2018-07-09 12:16:15 | INFO | Pointcal:  0319+415
2018-07-09 12:16:15 | INFO | Pointcal:  0319+415
2018-07-09 12:16:15 | INFO | Pointcal:  0319+415
2018-07-09 12:16:15 | INFO | Pointcal:  0319+415
2018-07-09 12:16:15 | INFO | Sources in MS all_avg.ms: 1302+5748,1252+5634,1331+305,1407+284,0319+415
2018-07-09 12:16:15 | INFO | Sources in MS all_avg.ms: 1302+5748,1252+5634,1331+305,1407+284,0319+415
2018-07-09 12:16:15 | INFO | Sources in MS all_avg.ms: 1302+5748,1252+5634,1331+305,1407+284,0319+415
2018-07-09 12:16:15 | INFO | Sources in MS all_avg.ms: 13

2018-07-09 12:16:17 | INFO | Previous calibration applied: ['delay.K1', 'allcal_p.G0', 'allcal_p_jitter.G0', 'bpcal_sp.B1']
2018-07-09 12:16:17 | INFO | Previous calibration applied: ['delay.K1', 'allcal_p.G0', 'allcal_p_jitter.G0', 'bpcal_sp.B1']
2018-07-09 12:16:17 | INFO | Previous calibration applied: ['delay.K1', 'allcal_p.G0', 'allcal_p_jitter.G0', 'bpcal_sp.B1']
2018-07-09 12:16:17 | INFO | Previous calibration applied: ['delay.K1', 'allcal_p.G0', 'allcal_p_jitter.G0', 'bpcal_sp.B1']
2018-07-09 12:16:17 | INFO | Previous calibration applied: ['delay.K1', 'allcal_p.G0', 'allcal_p_jitter.G0', 'bpcal_sp.B1']
2018-07-09 12:16:17 | INFO | Previous calibration applied: ['delay.K1', 'allcal_p.G0', 'allcal_p_jitter.G0', 'bpcal_sp.B1']
2018-07-09 12:16:17 | INFO | Previous calibration applied: ['delay.K1', 'allcal_p.G0', 'allcal_p_jitter.G0', 'bpcal_sp.B1']
2018-07-09 12:16:17 | INFO | Previous calibration applied: ['delay.K1', 'allcal_p.G0', 'allcal_p_jitter.G0', 'bpcal_sp.B1']
2018-07-

2018-07-09 12:17:02 | INFO | Amplitude amplitude calibration plot: ./plots/caltables/all_avg_allcal_ap.G3_amp.png
2018-07-09 12:17:02 | INFO | Amplitude amplitude calibration plot: ./plots/caltables/all_avg_allcal_ap.G3_amp.png
2018-07-09 12:17:02 | INFO | Amplitude amplitude calibration plot: ./plots/caltables/all_avg_allcal_ap.G3_amp.png
2018-07-09 12:17:02 | INFO | Amplitude amplitude calibration plot: ./plots/caltables/all_avg_allcal_ap.G3_amp.png
2018-07-09 12:17:02 | INFO | Deleted: ./calib/all_avg_allcal_ap_scan.G3
2018-07-09 12:17:02 | INFO | Deleted: ./calib/all_avg_allcal_ap_scan.G3
2018-07-09 12:17:02 | INFO | Deleted: ./calib/all_avg_allcal_ap_scan.G3
2018-07-09 12:17:02 | INFO | Deleted: ./calib/all_avg_allcal_ap_scan.G3
2018-07-09 12:17:02 | INFO | Deleted: ./calib/all_avg_allcal_ap_scan.G3
2018-07-09 12:17:02 | INFO | Deleted: ./calib/all_avg_allcal_ap_scan.G3
2018-07-09 12:17:02 | INFO | Deleted: ./calib/all_avg_allcal_ap_scan.G3
2018-07-09 12:17:02 | INFO | Deleted: ./

2018-07-09 12:17:02 | INFO | Generating calibration table: ./calib/all_avg_allcal_ap_scan.G3
2018-07-09 12:17:02 | INFO | Generating calibration table: ./calib/all_avg_allcal_ap_scan.G3
2018-07-09 12:17:02 | INFO | Generating calibration table: ./calib/all_avg_allcal_ap_scan.G3
2018-07-09 12:17:02 | INFO | Generating calibration table: ./calib/all_avg_allcal_ap_scan.G3
2018-07-09 12:17:02 | INFO | Generating calibration table: ./calib/all_avg_allcal_ap_scan.G3
2018-07-09 12:17:02 | INFO | Generating calibration table: ./calib/all_avg_allcal_ap_scan.G3
2018-07-09 12:17:02 | INFO | Generating calibration table: ./calib/all_avg_allcal_ap_scan.G3
2018-07-09 12:17:02 | INFO | Generating calibration table: ./calib/all_avg_allcal_ap_scan.G3
2018-07-09 12:17:02 | INFO | Generating calibration table: ./calib/all_avg_allcal_ap_scan.G3
2018-07-09 12:17:02 | INFO | Generating calibration table: ./calib/all_avg_allcal_ap_scan.G3
2018-07-09 12:17:02 | INFO | Generating calibration table: ./calib/all

({'Lo_dropout_scans': '',
  'applycal_all': 0,
  'average_1': 0,
  'bandpass_0': 0,
  'bandpass_1_sp': 0,
  'bpcal': '1407+284',
  'delay': 0,
  'first_images': 0,
  'fits_path': '/home/jupyter/data/',
  'flag_0_aoflagger': 0,
  'flag_1_apriori': 0,
  'flag_2a_manual': 0,
  'flag_2b_manual': 0,
  'flag_3_tfcropBP': 0,
  'flag_4_rflag': 0,
  'flag_statistics': 0,
  'fluxcal': '1331+305',
  'fluxscale': 0,
  'gain_0_p_ap': 0,
  'gain_1_amp_sp': 1,
  'hanning': 0,
  'inbase': 'all_avg',
  'init_models': 0,
  'manual_flags_a': './inputfg_a.flags',
  'manual_flags_b': './all_avg_1.flags',
  'ms2mms': 0,
  'phscals': '1302+5748',
  'plot_corrected': 0,
  'plot_data': 0,
  'ptcal': '0319+415',
  'refant': 'Mk2',
  'restore_flags': 0,
  'run_importfits': 0,
  'save_flags': 0,
  'shift_field_pos': 0,
  'summary_weblog': 0,
  'targets': '1252+5634',
  'weblog': 0},
 {'antennas': ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm'],
  'band': 'C',
  'baselines': ['Mk2&Kn',
   'Mk2&De',
   'Mk2&Pi',
   'Mk2&Da',

### 4.2.11 applycal_all
Two runs are executed. One to correct calibrators, using their own solutions when relevant. A second correction is executed for each target, using the solutions from the corresponding phase reference calibrator.

 - On calibrators: ['delay.K1','bpcal_sp.B1','allcal_p.G0','allcal_p_jitter.G0','allcal_ap.G3']
 - On targets: ['delay.K1','bpcal_sp.B1','phscal_p_scan.G2','allcal_ap_scan.G3']


In [29]:
inputs["gain_1_amp_sp"] = 0
inputs["applycal_all"] = 1

run_pipeline(inputs=inputs)

2018-07-09 12:17:28 | INFO | Starting pipeline
2018-07-09 12:17:28 | INFO | Starting pipeline
2018-07-09 12:17:28 | INFO | Starting pipeline
2018-07-09 12:17:28 | INFO | Starting pipeline
2018-07-09 12:17:28 | INFO | Starting pipeline
2018-07-09 12:17:28 | INFO | Starting pipeline
2018-07-09 12:17:28 | INFO | Starting pipeline
2018-07-09 12:17:28 | INFO | Starting pipeline
2018-07-09 12:17:28 | INFO | Starting pipeline
2018-07-09 12:17:28 | INFO | Starting pipeline
2018-07-09 12:17:28 | INFO | Starting pipeline
2018-07-09 12:17:28 | INFO | Starting pipeline
2018-07-09 12:17:28 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:17:28 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:17:28 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:17:28 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:17:28 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:17:28 | INFO | Running pipeline from: /hom

2018-07-09 12:17:28 | INFO | Fluxcal:   1331+305
2018-07-09 12:17:28 | INFO | Fluxcal:   1331+305
2018-07-09 12:17:28 | INFO | Fluxcal:   1331+305
2018-07-09 12:17:28 | INFO | Fluxcal:   1331+305
2018-07-09 12:17:28 | INFO | Fluxcal:   1331+305
2018-07-09 12:17:28 | INFO | Fluxcal:   1331+305
2018-07-09 12:17:28 | INFO | Bandpass:  1407+284
2018-07-09 12:17:28 | INFO | Bandpass:  1407+284
2018-07-09 12:17:28 | INFO | Bandpass:  1407+284
2018-07-09 12:17:28 | INFO | Bandpass:  1407+284
2018-07-09 12:17:28 | INFO | Bandpass:  1407+284
2018-07-09 12:17:28 | INFO | Bandpass:  1407+284
2018-07-09 12:17:28 | INFO | Bandpass:  1407+284
2018-07-09 12:17:28 | INFO | Bandpass:  1407+284
2018-07-09 12:17:28 | INFO | Bandpass:  1407+284
2018-07-09 12:17:28 | INFO | Bandpass:  1407+284
2018-07-09 12:17:28 | INFO | Bandpass:  1407+284
2018-07-09 12:17:28 | INFO | Bandpass:  1407+284
2018-07-09 12:17:28 | INFO | Pointcal:  0319+415
2018-07-09 12:17:28 | INFO | Pointcal:  0319+415
2018-07-09 12:17:28 

2018-07-09 12:17:31 | INFO | Previous calibration applied: ['delay.K1', 'bpcal_sp.B1', 'allcal_p.G0', 'allcal_p_jitter.G0', 'allcal_ap.G3']
2018-07-09 12:17:31 | INFO | Previous calibration applied: ['delay.K1', 'bpcal_sp.B1', 'allcal_p.G0', 'allcal_p_jitter.G0', 'allcal_ap.G3']
2018-07-09 12:17:31 | INFO | Previous calibration applied: ['delay.K1', 'bpcal_sp.B1', 'allcal_p.G0', 'allcal_p_jitter.G0', 'allcal_ap.G3']
2018-07-09 12:17:31 | INFO | Previous calibration applied: ['delay.K1', 'bpcal_sp.B1', 'allcal_p.G0', 'allcal_p_jitter.G0', 'allcal_ap.G3']
2018-07-09 12:17:31 | INFO | Previous calibration applied: ['delay.K1', 'bpcal_sp.B1', 'allcal_p.G0', 'allcal_p_jitter.G0', 'allcal_ap.G3']
2018-07-09 12:17:31 | INFO | Previous calibration applied: ['delay.K1', 'bpcal_sp.B1', 'allcal_p.G0', 'allcal_p_jitter.G0', 'allcal_ap.G3']
2018-07-09 12:17:31 | INFO | Previous calibration applied: ['delay.K1', 'bpcal_sp.B1', 'allcal_p.G0', 'allcal_p_jitter.G0', 'allcal_ap.G3']
2018-07-09 12:17:31 

2018-07-09 12:17:57 | INFO | Previous calibration applied: ['delay.K1', 'bpcal_sp.B1', 'phscal_p_scan.G2', 'allcal_ap_scan.G3']
2018-07-09 12:17:57 | INFO | Previous calibration applied: ['delay.K1', 'bpcal_sp.B1', 'phscal_p_scan.G2', 'allcal_ap_scan.G3']
2018-07-09 12:17:57 | INFO | Previous calibration applied: ['delay.K1', 'bpcal_sp.B1', 'phscal_p_scan.G2', 'allcal_ap_scan.G3']
2018-07-09 12:17:57 | INFO | Previous calibration gainfield: ['1302+5748', '1407+284', '1302+5748', '1302+5748']
2018-07-09 12:17:57 | INFO | Previous calibration gainfield: ['1302+5748', '1407+284', '1302+5748', '1302+5748']
2018-07-09 12:17:57 | INFO | Previous calibration gainfield: ['1302+5748', '1407+284', '1302+5748', '1302+5748']
2018-07-09 12:17:57 | INFO | Previous calibration gainfield: ['1302+5748', '1407+284', '1302+5748', '1302+5748']
2018-07-09 12:17:57 | INFO | Previous calibration gainfield: ['1302+5748', '1407+284', '1302+5748', '1302+5748']
2018-07-09 12:17:57 | INFO | Previous calibration g

({'Lo_dropout_scans': '',
  'applycal_all': 1,
  'average_1': 0,
  'bandpass_0': 0,
  'bandpass_1_sp': 0,
  'bpcal': '1407+284',
  'delay': 0,
  'first_images': 0,
  'fits_path': '/home/jupyter/data/',
  'flag_0_aoflagger': 0,
  'flag_1_apriori': 0,
  'flag_2a_manual': 0,
  'flag_2b_manual': 0,
  'flag_3_tfcropBP': 0,
  'flag_4_rflag': 0,
  'flag_statistics': 0,
  'fluxcal': '1331+305',
  'fluxscale': 0,
  'gain_0_p_ap': 0,
  'gain_1_amp_sp': 0,
  'hanning': 0,
  'inbase': 'all_avg',
  'init_models': 0,
  'manual_flags_a': './inputfg_a.flags',
  'manual_flags_b': './all_avg_1.flags',
  'ms2mms': 0,
  'phscals': '1302+5748',
  'plot_corrected': 0,
  'plot_data': 0,
  'ptcal': '0319+415',
  'refant': 'Mk2',
  'restore_flags': 0,
  'run_importfits': 0,
  'save_flags': 0,
  'shift_field_pos': 0,
  'summary_weblog': 0,
  'targets': '1252+5634',
  'weblog': 0},
 {'antennas': ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm'],
  'applycal_all': True,
  'band': 'C',
  'baselines': ['Mk2&Kn',
   'Mk2&De',
 

### 4.2.12 flag_4_rflag

After calibration, we can run `flagdata` in rflag mode. This mode requires the data to be already calibrated.

CASA `flagdata`

|Parameter      | Value    |
|-------------- | -------- |
| mode          | rflag    |
| correlation   | ABS_ALL  |
| ntime         | 90min    |
| combinescans  | True     |
| datacolumn    | corrected|
| timedevscale  | 5        |
| freqdevscale  | 5        |
| action        | apply    |


In [30]:
inputs["applycal_all"] = 0
inputs["flag_4_rflag"] = 1

run_pipeline(inputs=inputs)

2018-07-09 12:18:24 | INFO | Starting pipeline
2018-07-09 12:18:24 | INFO | Starting pipeline
2018-07-09 12:18:24 | INFO | Starting pipeline
2018-07-09 12:18:24 | INFO | Starting pipeline
2018-07-09 12:18:24 | INFO | Starting pipeline
2018-07-09 12:18:24 | INFO | Starting pipeline
2018-07-09 12:18:24 | INFO | Starting pipeline
2018-07-09 12:18:24 | INFO | Starting pipeline
2018-07-09 12:18:24 | INFO | Starting pipeline
2018-07-09 12:18:24 | INFO | Starting pipeline
2018-07-09 12:18:24 | INFO | Starting pipeline
2018-07-09 12:18:24 | INFO | Starting pipeline
2018-07-09 12:18:24 | INFO | Starting pipeline
2018-07-09 12:18:24 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:18:24 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:18:24 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:18:24 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:18:24 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-0

2018-07-09 12:18:24 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:24 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:24 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:24 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:24 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:24 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:24 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:24 | INFO | Fluxcal:   1331+305
2018-07-09 12:18:24 | INFO | Fluxcal:   1331+305
2018-07-09 12:18:24 | INFO | Fluxcal:   1331+305
2018-07-09 12:18:24 | INFO | Fluxcal:   1331+305
2018-07-09 12:18:24 | INFO | Fluxcal:   1331+305
2018-07-09 12:18:24 | INFO | Fluxcal:   1331+305
2018-07-09 12:18:24 | INFO | Fluxcal:   1331+305
2018-07-09 12:18:24 | INFO | Fluxcal:   1331+305
2018-07-09 12:18:24 | INFO | Fluxcal:   1331+305
2018-07-09 12:18:24 | INFO | Fluxcal:   1331+305
2018-07-09 12:18:24 | INFO | Fluxcal:   1331+305
2018-07-09 12:18:24 | INFO | Fluxcal:   1331+305
2018-07-09 12:18:24 | INFO | Fluxcal:   1331+305
2018-07-09 12

2018-07-09 12:18:27 | INFO | Pipeline finished
2018-07-09 12:18:27 | INFO | Pipeline finished
2018-07-09 12:18:27 | INFO | Pipeline finished
2018-07-09 12:18:27 | INFO | Pipeline finished
2018-07-09 12:18:27 | INFO | Pipeline finished
2018-07-09 12:18:27 | INFO | #################
2018-07-09 12:18:27 | INFO | #################
2018-07-09 12:18:27 | INFO | #################
2018-07-09 12:18:27 | INFO | #################
2018-07-09 12:18:27 | INFO | #################
2018-07-09 12:18:27 | INFO | #################
2018-07-09 12:18:27 | INFO | #################
2018-07-09 12:18:27 | INFO | #################
2018-07-09 12:18:27 | INFO | #################
2018-07-09 12:18:27 | INFO | #################
2018-07-09 12:18:27 | INFO | #################
2018-07-09 12:18:27 | INFO | #################
2018-07-09 12:18:27 | INFO | #################


({'Lo_dropout_scans': '',
  'applycal_all': 0,
  'average_1': 0,
  'bandpass_0': 0,
  'bandpass_1_sp': 0,
  'bpcal': '1407+284',
  'delay': 0,
  'first_images': 0,
  'fits_path': '/home/jupyter/data/',
  'flag_0_aoflagger': 0,
  'flag_1_apriori': 0,
  'flag_2a_manual': 0,
  'flag_2b_manual': 0,
  'flag_3_tfcropBP': 0,
  'flag_4_rflag': 1,
  'flag_statistics': 0,
  'fluxcal': '1331+305',
  'fluxscale': 0,
  'gain_0_p_ap': 0,
  'gain_1_amp_sp': 0,
  'hanning': 0,
  'inbase': 'all_avg',
  'init_models': 0,
  'manual_flags_a': './inputfg_a.flags',
  'manual_flags_b': './all_avg_1.flags',
  'ms2mms': 0,
  'phscals': '1302+5748',
  'plot_corrected': 0,
  'plot_data': 0,
  'ptcal': '0319+415',
  'refant': 'Mk2',
  'restore_flags': 0,
  'run_importfits': 0,
  'save_flags': 0,
  'shift_field_pos': 0,
  'summary_weblog': 0,
  'targets': '1252+5634',
  'weblog': 0},
 {'antennas': ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm'],
  'band': 'C',
  'baselines': ['Mk2&Kn',
   'Mk2&De',
   'Mk2&Pi',
   'Mk2&Da',

### 4.2.13 plot_corrected

Produce plots of amp/phase vs time/freq for each baseline	plotms using corrected data column.

Inputs parameters needed:
```
inbase                  [str]
targets                 [str]
phscals                 [str]
fluxcal                 [str]
bpcal                   [str]
ptcal                   [str]
```

Output:
```
./plots/plots_corrected [directory with multiple plots in png format]
```

Produces plots in png format for the visibilities in `inbase_avg.ms`. It iterates through all sources specified by the user in the inputs file, and produces plots for each baseline. All plots are stored in the output directory. The plots have some averaging:

 - Amp/Phase vs Time: all channels averaged in each spw, colorized by spw.
 - Amp/Phase vs Frequency: averaged every 5 min, colorized by correlation.


In [31]:
inputs["flag_4_rflag"] = 0
inputs["plot_corrected"] = 1

run_pipeline(inputs=inputs)

2018-07-09 12:18:27 | INFO | Starting pipeline
2018-07-09 12:18:27 | INFO | Starting pipeline
2018-07-09 12:18:27 | INFO | Starting pipeline
2018-07-09 12:18:27 | INFO | Starting pipeline
2018-07-09 12:18:27 | INFO | Starting pipeline
2018-07-09 12:18:27 | INFO | Starting pipeline
2018-07-09 12:18:27 | INFO | Starting pipeline
2018-07-09 12:18:27 | INFO | Starting pipeline
2018-07-09 12:18:27 | INFO | Starting pipeline
2018-07-09 12:18:27 | INFO | Starting pipeline
2018-07-09 12:18:27 | INFO | Starting pipeline
2018-07-09 12:18:27 | INFO | Starting pipeline
2018-07-09 12:18:27 | INFO | Starting pipeline
2018-07-09 12:18:27 | INFO | Starting pipeline
2018-07-09 12:18:27 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:18:27 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:18:27 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:18:27 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:18:27 | INFO | Running p

2018-07-09 12:18:27 | INFO | Targets:   1252+5634
2018-07-09 12:18:27 | INFO | Targets:   1252+5634
2018-07-09 12:18:27 | INFO | Targets:   1252+5634
2018-07-09 12:18:27 | INFO | Targets:   1252+5634
2018-07-09 12:18:27 | INFO | Targets:   1252+5634
2018-07-09 12:18:27 | INFO | Targets:   1252+5634
2018-07-09 12:18:27 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:27 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:27 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:27 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:27 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:27 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:27 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:27 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:27 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:27 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:27 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:27 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:27 | INFO | Phasecals: 1302+5748
2018-07-09 12:18:27 | INFO | Phasecals: 1302+5748


2018-07-09 12:18:30 | INFO | Producing in parallel visibility plots for fields: 0319+415,1331+305,1302+5748,1252+5634,1407+284
2018-07-09 12:18:30 | INFO | Producing in parallel visibility plots for fields: 0319+415,1331+305,1302+5748,1252+5634,1407+284
2018-07-09 12:18:30 | INFO | Producing in parallel visibility plots for fields: 0319+415,1331+305,1302+5748,1252+5634,1407+284
2018-07-09 12:18:30 | INFO | Producing in parallel visibility plots for fields: 0319+415,1331+305,1302+5748,1252+5634,1407+284
2018-07-09 12:18:30 | INFO | Producing in parallel visibility plots for fields: 0319+415,1331+305,1302+5748,1252+5634,1407+284
2018-07-09 12:18:30 | INFO | Visibility plots for field: 0319+415, datacolumn: corrected
2018-07-09 12:18:30 | INFO | Visibility plots for field: 0319+415, datacolumn: corrected
2018-07-09 12:18:30 | INFO | Visibility plots for field: 0319+415, datacolumn: corrected
2018-07-09 12:18:30 | INFO | Visibility plots for field: 0319+415, datacolumn: corrected
2018-07-0

2018-07-09 12:20:03 | INFO | Finished field: 1302+5748, output: ./plots/plots_corrected/all_avg_4plot_1302+5748_corrected{0-4}.png
2018-07-09 12:20:03 | INFO | Finished field: 1302+5748, output: ./plots/plots_corrected/all_avg_4plot_1302+5748_corrected{0-4}.png
2018-07-09 12:20:03 | INFO | Finished field: 1302+5748, output: ./plots/plots_corrected/all_avg_4plot_1302+5748_corrected{0-4}.png
2018-07-09 12:20:03 | INFO | Finished field: 1302+5748, output: ./plots/plots_corrected/all_avg_4plot_1302+5748_corrected{0-4}.png
2018-07-09 12:20:03 | INFO | Finished field: 1302+5748, output: ./plots/plots_corrected/all_avg_4plot_1302+5748_corrected{0-4}.png
2018-07-09 12:20:03 | INFO | Finished field: 1302+5748, output: ./plots/plots_corrected/all_avg_4plot_1302+5748_corrected{0-4}.png
2018-07-09 12:20:03 | INFO | Finished field: 1302+5748, output: ./plots/plots_corrected/all_avg_4plot_1302+5748_corrected{0-4}.png
2018-07-09 12:20:03 | INFO | Finished field: 1302+5748, output: ./plots/plots_corre

2018-07-09 12:22:02 | INFO | Visibility plots finished
2018-07-09 12:22:02 | INFO | Visibility plots finished
2018-07-09 12:22:02 | INFO | Visibility plots finished
2018-07-09 12:22:02 | INFO | Visibility plots finished
2018-07-09 12:22:02 | INFO | Visibility plots finished
2018-07-09 12:22:02 | INFO | End plot_corrected
2018-07-09 12:22:02 | INFO | End plot_corrected
2018-07-09 12:22:02 | INFO | End plot_corrected
2018-07-09 12:22:02 | INFO | End plot_corrected
2018-07-09 12:22:02 | INFO | End plot_corrected
2018-07-09 12:22:02 | INFO | End plot_corrected
2018-07-09 12:22:02 | INFO | End plot_corrected
2018-07-09 12:22:02 | INFO | End plot_corrected
2018-07-09 12:22:02 | INFO | End plot_corrected
2018-07-09 12:22:02 | INFO | End plot_corrected
2018-07-09 12:22:02 | INFO | End plot_corrected
2018-07-09 12:22:02 | INFO | End plot_corrected
2018-07-09 12:22:02 | INFO | End plot_corrected
2018-07-09 12:22:02 | INFO | End plot_corrected
2018-07-09 12:22:02 | INFO | Create directory: ./plot

2018-07-09 12:27:35 | INFO | Pipeline finished
2018-07-09 12:27:35 | INFO | Pipeline finished
2018-07-09 12:27:35 | INFO | Pipeline finished
2018-07-09 12:27:35 | INFO | Pipeline finished
2018-07-09 12:27:35 | INFO | Pipeline finished
2018-07-09 12:27:35 | INFO | Pipeline finished
2018-07-09 12:27:35 | INFO | Pipeline finished
2018-07-09 12:27:35 | INFO | Pipeline finished
2018-07-09 12:27:35 | INFO | Pipeline finished
2018-07-09 12:27:35 | INFO | Pipeline finished
2018-07-09 12:27:35 | INFO | #################
2018-07-09 12:27:35 | INFO | #################
2018-07-09 12:27:35 | INFO | #################
2018-07-09 12:27:35 | INFO | #################
2018-07-09 12:27:35 | INFO | #################
2018-07-09 12:27:35 | INFO | #################
2018-07-09 12:27:35 | INFO | #################
2018-07-09 12:27:35 | INFO | #################
2018-07-09 12:27:35 | INFO | #################
2018-07-09 12:27:35 | INFO | #################
2018-07-09 12:27:35 | INFO | #################
2018-07-09 12

({'Lo_dropout_scans': '',
  'applycal_all': 0,
  'average_1': 0,
  'bandpass_0': 0,
  'bandpass_1_sp': 0,
  'bpcal': '1407+284',
  'delay': 0,
  'first_images': 0,
  'fits_path': '/home/jupyter/data/',
  'flag_0_aoflagger': 0,
  'flag_1_apriori': 0,
  'flag_2a_manual': 0,
  'flag_2b_manual': 0,
  'flag_3_tfcropBP': 0,
  'flag_4_rflag': 0,
  'flag_statistics': 0,
  'fluxcal': '1331+305',
  'fluxscale': 0,
  'gain_0_p_ap': 0,
  'gain_1_amp_sp': 0,
  'hanning': 0,
  'inbase': 'all_avg',
  'init_models': 0,
  'manual_flags_a': './inputfg_a.flags',
  'manual_flags_b': './all_avg_1.flags',
  'ms2mms': 0,
  'phscals': '1302+5748',
  'plot_corrected': 1,
  'plot_data': 0,
  'ptcal': '0319+415',
  'refant': 'Mk2',
  'restore_flags': 0,
  'run_importfits': 0,
  'save_flags': 0,
  'shift_field_pos': 0,
  'summary_weblog': 0,
  'targets': '1252+5634',
  'weblog': 0},
 {'antennas': ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm'],
  'band': 'C',
  'baselines': ['Mk2&Kn',
   'Mk2&De',
   'Mk2&Pi',
   'Mk2&Da',

### 4.2.14 weblog
Update the weblog will all available information and plots.

As the initial `summary_weblog` but the weblog will include all information related to the dataset, the observation, the calibration and the visibilities. There are four different web pages:

 - Home. Basic dataset information (name, date, antennas, frequency, averaging, etc.)
 - Observation summary. Access to listobs, antennas, elevation plot, uvcov plots per each source.
 - Calibration. List of tables produced and plots showing the solutions.
 - Plots. Amp/phase vs time/freq per source and per baseline. Corrected and uncorrected visibilities, and Amp/phase vs uvdist. Also statistics of flagged visibilities per scan, field, correlation, spw and antenna.

In [32]:
inputs["plot_corrected"] = 0
inputs["weblog"] = 1

run_pipeline(inputs=inputs)

2018-07-09 12:33:24 | INFO | Starting pipeline
2018-07-09 12:33:24 | INFO | Starting pipeline
2018-07-09 12:33:24 | INFO | Starting pipeline
2018-07-09 12:33:24 | INFO | Starting pipeline
2018-07-09 12:33:24 | INFO | Starting pipeline
2018-07-09 12:33:24 | INFO | Starting pipeline
2018-07-09 12:33:24 | INFO | Starting pipeline
2018-07-09 12:33:24 | INFO | Starting pipeline
2018-07-09 12:33:24 | INFO | Starting pipeline
2018-07-09 12:33:24 | INFO | Starting pipeline
2018-07-09 12:33:24 | INFO | Starting pipeline
2018-07-09 12:33:24 | INFO | Starting pipeline
2018-07-09 12:33:24 | INFO | Starting pipeline
2018-07-09 12:33:24 | INFO | Starting pipeline
2018-07-09 12:33:24 | INFO | Starting pipeline
2018-07-09 12:33:24 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:33:24 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:33:24 | INFO | Running pipeline from: /home/jupyter/pipeline/
2018-07-09 12:33:24 | INFO | Running pipeline from: /home/jupyter/p

2018-07-09 12:33:24 | INFO | Reading ms file information for MS: all_avg.ms
2018-07-09 12:33:24 | INFO | Reading ms file information for MS: all_avg.ms
2018-07-09 12:33:24 | INFO | Targets:   1252+5634
2018-07-09 12:33:24 | INFO | Targets:   1252+5634
2018-07-09 12:33:24 | INFO | Targets:   1252+5634
2018-07-09 12:33:24 | INFO | Targets:   1252+5634
2018-07-09 12:33:24 | INFO | Targets:   1252+5634
2018-07-09 12:33:24 | INFO | Targets:   1252+5634
2018-07-09 12:33:24 | INFO | Targets:   1252+5634
2018-07-09 12:33:24 | INFO | Targets:   1252+5634
2018-07-09 12:33:24 | INFO | Targets:   1252+5634
2018-07-09 12:33:24 | INFO | Targets:   1252+5634
2018-07-09 12:33:24 | INFO | Targets:   1252+5634
2018-07-09 12:33:24 | INFO | Targets:   1252+5634
2018-07-09 12:33:24 | INFO | Targets:   1252+5634
2018-07-09 12:33:24 | INFO | Targets:   1252+5634
2018-07-09 12:33:24 | INFO | Targets:   1252+5634
2018-07-09 12:33:24 | INFO | Phasecals: 1302+5748
2018-07-09 12:33:24 | INFO | Phasecals: 1302+574

2018-07-09 12:33:27 | INFO | Loaded previous calibration tables from: ./calib/caltables.pkl
2018-07-09 12:33:27 | INFO | Loaded previous calibration tables from: ./calib/caltables.pkl
2018-07-09 12:33:27 | INFO | Loaded previous calibration tables from: ./calib/caltables.pkl
2018-07-09 12:33:27 | INFO | Loaded previous calibration tables from: ./calib/caltables.pkl
2018-07-09 12:33:27 | INFO | Loaded previous calibration tables from: ./calib/caltables.pkl
2018-07-09 12:33:27 | INFO | Loaded previous calibration tables from: ./calib/caltables.pkl
2018-07-09 12:33:27 | INFO | Elevation plot found.
2018-07-09 12:33:27 | INFO | Elevation plot found.
2018-07-09 12:33:27 | INFO | Elevation plot found.
2018-07-09 12:33:27 | INFO | Elevation plot found.
2018-07-09 12:33:27 | INFO | Elevation plot found.
2018-07-09 12:33:27 | INFO | Elevation plot found.
2018-07-09 12:33:27 | INFO | Elevation plot found.
2018-07-09 12:33:27 | INFO | Elevation plot found.
2018-07-09 12:33:27 | INFO | Elevation p

({'Lo_dropout_scans': '',
  'applycal_all': 0,
  'average_1': 0,
  'bandpass_0': 0,
  'bandpass_1_sp': 0,
  'bpcal': '1407+284',
  'delay': 0,
  'first_images': 0,
  'fits_path': '/home/jupyter/data/',
  'flag_0_aoflagger': 0,
  'flag_1_apriori': 0,
  'flag_2a_manual': 0,
  'flag_2b_manual': 0,
  'flag_3_tfcropBP': 0,
  'flag_4_rflag': 0,
  'flag_statistics': 0,
  'fluxcal': '1331+305',
  'fluxscale': 0,
  'gain_0_p_ap': 0,
  'gain_1_amp_sp': 0,
  'hanning': 0,
  'inbase': 'all_avg',
  'init_models': 0,
  'manual_flags_a': './inputfg_a.flags',
  'manual_flags_b': './all_avg_1.flags',
  'ms2mms': 0,
  'phscals': '1302+5748',
  'plot_corrected': 0,
  'plot_data': 0,
  'ptcal': '0319+415',
  'refant': 'Mk2',
  'restore_flags': 0,
  'run_importfits': 0,
  'save_flags': 0,
  'shift_field_pos': 0,
  'summary_weblog': 0,
  'targets': '1252+5634',
  'weblog': 1},
 {'antennas': ['Mk2', 'Pi', 'Da', 'Kn', 'De', 'Cm'],
  'band': 'C',
  'baselines': ['Mk2&Kn',
   'Mk2&De',
   'Mk2&Pi',
   'Mk2&Da',